## Arrays & Hashing

### Medium

[665. Non-decreasing Array](https://leetcode.com/problems/non-decreasing-array/)  

`nums[i] > nums[i + 1]`, fix it:
- If `nums[i - 1] > nums[i + 1]`,  increase `nums[i + 1]` to keep the array consistent.
- else decrease `nums[i]`. (no need bc no comparison made with i anymore)

#2,5,4, 4 ->#2-4-44  
#2,5,4, 5 -> 2-4-45  
#423 ->223
#if i+1 < i-1 -> i+1 = i
but 3425 -> 3-2-25, not right, 34-4-5 right ( 3225)

In [ ]:
def checkPossibility(nums: list[int]) -> bool:
    m = 0 #number to modify
    for i in range(len(nums)-1):
        if nums[i] > nums[i+1]:
            if m == 1: 
                return False
            m+=1
            if nums[i+1] < nums[i-1] and i>0:
                nums[i+1] = nums[i]

    return m<=1

checkPossibility([3,4,2,3])
checkPossibility([1,4,1,2])

[1, 1, 1, 2]


True

[49. Group Anagrams](https://leetcode.com/problems/group-anagrams/)

In [ ]:
from collections import defaultdict
def groupAnagrams(strs: list[str]) -> list[list[str]]:
    res = defaultdict(list) #value is list, key is tuple, e.g key-value: (1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['eat', 'tea', 'ate']

    for s in strs:
        count = [0] * 26
        for c in s:
            count[ord(c) - ord('a')] += 1
        res[tuple(count)].append(s)
        print( tuple(count), res[tuple(count)])
    return list(res.values())
'''
Time : O(m∗n) - Space : O(m)
 m is the number of strings and n is length of longest string.
'''
strs = ["eat","tea","tan","ate","nat","bat"]
groupAnagrams(strs)

(1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['eat']
(1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['eat', 'tea']
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['tan']
(1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['eat', 'tea', 'ate']
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['tan', 'nat']
(1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0) ['bat']


[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

[5. Longest Palindromic Substring](https://leetcode.com/problems/longest-palindromic-substring/?envType=featured-list&envId=top-interview-questions?envType=featured-list&envId=top-interview-questions)


Manacher most effiencient

In **Manacher’s Algorithm**, the step

```python
if i < R:
    p[i] = min(R - i, p[mirr])
```

ensures we have a **good initial radius** for expanding the palindrome at index `i` without **overestimating**. The rationale can be understood by focusing on two key ideas:

1. **Using the Mirror’s Radius**: We reuse information about a palindrome that is symmetric (mirrored) around the current center.
2. **Not Exceeding the Known Boundary**: We cannot trust a radius larger than `R - i` because we only know for sure that the palindrome is valid **inside** the current right boundary (`R`).

Let’s walk through **why** this formula works, step by step, and illustrate it with a simple scenario.

---

 The Manacher’s Setup

- We have a **current palindrome** with **center** = `C` and a **right boundary** = `R`.  
- We are processing a new position `i` such that `i < R` (i.e., **inside** that known palindrome).  
- We define the **mirror** of `i` around `C` as `mirr = 2*C - i`. 

In simpler terms, `mirr` is the **equidistant position** on the left side of `C` that “matches up” with `i` on the right side.

---

 Two Main Cases

When we say

```python
p[i] = min(R - i, p[mirr])
```
we’re combining two considerations:

1. **`p[mirr]`**: The palindrome radius at the mirror index might give a clue how large the palindrome at `i` can be without further checks.  
2. **`R - i`**: We can’t exceed the known boundary `R`. If `p[mirr]` is bigger than `(R - i)`, that portion would fall outside the current palindrome’s boundary, and we don’t yet know if the palindrome at `i` extends beyond `R`.

Hence we pick the smaller of these two as an **initial** radius. If the palindrome truly extends further, we’ll find out in the subsequent expansion step:

```python
while t[i + p[i] + 1] == t[i - p[i] - 1]:
    p[i] += 1
```

---

 Case A: `p[mirr] <= R - i`

- The palindrome at the mirror doesn’t exceed the left boundary of the big palindrome.  
- We can safely **copy** `p[mirr]` over to `p[i]` because we know the palindrome at `i` can at least be that large—no boundary conflict arises.

**Example Visual**  
```
      C             R
...|---|-----|-------|...
      |   i   |
      | mirr  |
      
If p[mirr] is small enough that it fits wholly inside the boundary,
we just set p[i] = p[mirr].
```

 Case B: `p[mirr] > R - i`

- The palindrome at the mirror **would cross** the boundary if we blindly copied it to `p[i]`.  
- We must limit `p[i]` to `R - i`. This is all we **know** for certain without expanding further because it doesn’t exceed the known right boundary.

**Example Visual**  
```
          C        R
...|-------|-------|...
         i    ?

If p[mirr] suggests an even bigger palindrome,
we still can’t trust it beyond R, so we set p[i] = R - i.
```

---

Concrete Example

Let’s take a **transformed** string (as Manacher’s uses extra characters like `#`) for `"aba"`, say `^#a#b#a#$`. Suppose:

- `C = 4` (around `#b#`),  
- `R = 6`, meaning we know the palindrome extends to position `6`.

Now we’re calculating for `i = 5`, which is inside the boundary (`5 < R`):

1. `mirr = 2*C - i = 2*4 - 5 = 3`.  
2. Suppose `p[mirr] = 3`, but `(R - i) = 1`.  
   - This means if we blindly set `p[i] = p[mirr] = 3`, we’d be claiming a palindrome extends to `i + 3 = 8`, beyond our known boundary `6`.  
   - So we must do `p[i] = min(p[mirr], R - i) = min(3, 1) = 1`.  
   - After that, we attempt to expand further if the characters still match.

This approach **never overestimates** the radius, ensuring correctness. If in reality the palindrome does extend beyond the boundary, the **while loop** expansion will catch that. Otherwise, we keep the valid radius limited by `(R - i)`.

---

- **No Over-Expansion**: We never assume beyond what’s known from symmetry plus the boundary.  
- **Reuse of Computed Palindromes**: We gain efficiency because we don’t re-check everything from scratch.  
- **Subsequent Expansion**: If the palindrome is indeed larger, the next expansions step will increment `p[i]` further.

Hence, `p[i] = min(R - i, p[mirr])` is the perfect initial guess for the palindrome radius at `i`, ensuring we remain within the known boundary while leveraging mirrored data from `mirr`.

---

In Summary

1. **Mirror** (`mirr`) tells us how big the palindrome was on the symmetric left side.  
2. **`R - i`** ensures we don’t cross the known right boundary.  
3. The **minimum** of these two values is used as a **starting radius** (not the final). We expand afterwards if possible.  

That’s how **Manacher’s** obtains a **linear-time** palindrome check without redoing expansions that were already implied by the mirrored position.

In [ ]:
def longestPalindrome( s: str) -> str:
      # Preprocess the string with special characters
    t = '#'.join('^{}$'.format(s))
    n = len(t)
    p = [0] * n
    C = R = 0
    maxLen, maxCenter = 0, 0
    for i in range(1, n - 1):

        mirr = 2 * C - i
        #This part starts working when C <= i < R,
        # get p[i] use the mirror trick
        if i < R:
            p[i] = min(R - i, p[mirr])
        
        #2. Keep expand around i to check palindrome
        while t[i + p[i] + 1] == t[i - p[i] - 1]:
            p[i] += 1
        #3. Extend R and C as p[i] has Pal value
        if i + p[i] > R:
            C, R = i, i + p[i]
        
        #update maxLen
        if maxLen < p[i]:
            maxLen = p[i]
            maxCenter = i
    return s[(maxCenter - maxLen) // 2: (maxCenter + maxLen) // 2]

Fastest Solution: only check next longer pal. 
- Loop through the string, r is bounded at index (r = i + 1)
    - odd length check:
        l = r - (res_len + 2)
        
    

In [1]:
def longestPalindrome(self, s: str) -> str:
    if s == s[::-1]:
        return s

    res_L = 0
    res_len = 1

    for i in range(1, len(s)):
        r  = i + 1

        # Odd length palindrome
        l = r - (res_len+2)
        substring = s[l:r]
        # l can be negative, so check if it is >= 0
        if l >= 0 and substring == substring[::-1]:
            res_L = l
            res_len = len(substring)
            # The even wil be shorter, so no need to compute it
            continue

        # Even length palindrome
        l = r - (res_len+1)
        substring = s[l:r]
        # l can be negative, so check if it is >= 0
        if l >= 0 and substring == substring[::-1]:
            res_L = l
            res_len = len(substring)
            
    return s[res_L:res_L + res_len]
longestPalindrome("abaxabaxabb")


'baxabaxab'

- Loop through the whole strings. check palin that going outwards at each element.
- At each element, expand in 2 ways to check odd length pal, and even length. 
    - To check odd length, starts R = 1
    - To check even length, R = L + 1

In [2]:
#Neetcode, not optimal
def longestPalindrome(s: str) -> str:
        #for each letter, expands outwards        
        resL, resR = 0,0

        for i in range(len(s)):
            #odd length
            l,r = i,i
            while l >= 0 and r < len(s) and s[l] == s[r]:
                #update before moving outwards
                if max(resR - resL + 1, r - l + 1 ) == r - l + 1:
                    resR, resL = r, l
                l-=1
                r+=1
            #even length
            l, r = i, i+1
            #exact same move outward code like above
            while l >= 0 and r < len(s) and s[l] == s[r]:
                if max(resR - resL + 1, r - l + 1) == r - l +1 :
                    resR, resL = r, l
                l -=1
                r +=1
        return s[resL: resR + 1]
s = "babad"
longestPalindrome(s)

'aba'

Longest Palindrome with lexicographical smallest

In [ ]:
def longestPalindrome(s):
  # Preprocess the string to handle both even and odd-length palindromes
    t = '#'.join('^{}$'.format(s))  # Add boundaries and separators
    n = len(t)
    p = [0] * n  # Array to store palindrome radii
    center = right = 0  # Center and right boundary of the current longest palindrome
    max_len = 0  # Maximum length of palindrome found
    longest_palindrome = ""  # Longest palindrome found so far

    for i in range(1, n - 1):
        # Mirror of the current position `i` with respect to center `C`
        mirror = 2 * center - i
        
        # Use previously computed palindrome length if within boundary
        if i < right:
            p[i] = min(right - i, p[mirror])
        
        # Expand around center `i`
        while t[i + p[i] + 1] == t[i - p[i] - 1]:
            p[i] += 1
        
        # Update center and right boundary if needed
        if i + p[i] > right:
            center, right = i, i + p[i]
        
        # Calculate the start and end indices of the palindrome in the original string
        length = p[i]
        start = (i - length) // 2
        end = start + length

        # Check if the new palindrome is longer or lexicographically smaller
        current_palindrome = s[start:end]
        if length > max_len or (length == max_len and current_palindrome < longest_palindrome):
            max_len = length
            longest_palindrome = current_palindrome

    return longest_palindrome

s = "abacdfgdcabba"
print(longestPalindrome(s))  # Output: "abba"


[36. Valid Sudoku](https://leetcode.com/problems/valid-sudoku/)- defaultdict(set)

In [54]:
from collections import defaultdict 
#36. Valid Sudoku
def isValidSudoku(board: list[list[str]]) -> bool:
    #Idea: check if each small square belongs to coordinated-associated rows, cols, and sqr 
    #Implement: create hashset (dict of sets) as cols, rows and sqr
    #each row, col, sqr is a set
    #Form of each dict(set): e.g: rows = {0: {set}, 1:{set},...}
    cols = defaultdict(set) 
    rows = defaultdict(set)
    sqr = defaultdict(set)

    for r in range(9):
            for c in range(9):
                if board[r][c] == ".":
                    continue
                if (
                    board[r][c] in rows[r]
                    or board[r][c] in cols[c]
                    or board[r][c] in sqr[(r // 3, c // 3)]
                ):
                    return False
                cols[c].add(board[r][c])
                rows[r].add(board[r][c])
                sqr[(r // 3, c // 3)].add(board[r][c])

    return True

[238. Product of Array Except Self (2) - Only pos var](https://leetcode.com/problems/product-of-array-except-self/submissions/1089079628/)

In [ ]:
def productExceptSelf(nums: list[int]) -> list[int]:
    #Create an preArr stores prefix product up to + (exclude) nums[i]
    #Loop from the end of preArr: at each preArr[i], multiply by current postfix product var

    #Time: O(n), Space: O(n)
    out = [1] * len(nums)
    
    #Create an arr stores prefix product up to nums[i] + (exclude) nums[i]
    for i in range(1, len(nums)):
        out[i] = out[i-1] * nums[i-1]
    #init first and last in output
    pos = nums[-1]
    for i in range(len(nums)-2, -1, -1):
        out[i] = out[i] * pos  #upto prefix * upto postfix 
        pos *= nums[i]

    return out
nums = [1,2,3,4]
productExceptSelf(nums)

[24, 12, 8, 6]

[238. Product of Array Except Self - Pre and pos as arrays](https://leetcode.com/problems/product-of-array-except-self/submissions/1089034138/)

In [3]:
def productExceptSelf(nums: list[int]) -> list[int]:
    #Create 2 arrays: prefix product and postfix product.
    #Output[i] = prefix[i-1] * postfix [i+1]
    #Time: O(n), Space: O(3n)
    l = len(nums)
    pre, pos= [nums[0]]*l, [nums[l-1]]*l
    j = l - 2
    for i in range(1, l):
        pre[i] = pre[i-1] * nums[i]
        pos[j] = pos[j+1] * nums[j]
        j-=1
    #init first and last in output
    out = [pos[1]]*l
    out[l-1] = pre[l-2]

    for i in range(1,l-1):
        out[i] = pre[i-1]*pos[i+1]

    print(pre, pos, out)
    return out

nums = [1,2,3,4]
productExceptSelf(nums)

[1, 2, 6, 24] [24, 24, 12, 4] [24, 12, 8, 6]


[24, 12, 8, 6]

[128. Longest Consecutive Sequence. Original](https://leetcode.com/problems/longest-consecutive-sequence/)

If a number is part of a sequence, it'll visited at most twice. If a number is only a sequence of itself, it'll be visited once. So time complexity = O(2n) = O(n).
Use a set

In [60]:
def longestConsecutive(nums: list[int]) -> int:
    numSet = set(nums)
    longest = 0
    for num in numSet:  
        #check if num is the start of the seq:
        if (num - 1) not in numSet:
            len = 0
            while (num+len) in numSet:
                len+=1
            longest = max(longest,len)

    return longest
nums = [9,1,4,7,3,-1,0,5,8,-1,6]
longestConsecutive(nums)


21


7

[347. Top K Frequent Elements ](https://leetcode.com/problems/top-k-frequent-elements/submissions/)

Bucket sort - Space: Linear, Time: Linear


In [68]:
def topKFrequent(nums: list[int], k: int) -> list[int]:
#bucket sort: create an array where index is the freqs, value is list of nums associating with the freqs
# Space: O(N). Size of bucket always <= len(nums) because max freq of any num is len(nums).
    count = {} # count dict
    
    buckets = [[] for i in range (len(nums)+1)] #freqs start from 1 to including len(nums)

    #manually count instead of collections.Counter(nums)
    for n in nums:
        count[n] = 1 + count.get(n, 0) #count.get(n, 0) means count[n] = 0 if n has not exist in dict
    
    #bucket's index is the freqs, value is list of nums associating with the freqs
    for num,freq in count.items():
        
        buckets[freq].append(num)
    res = []
    for i in range(len(buckets)-1, 0, -1): #start from last index
        if len(buckets[i]) == 0:
            continue
        for num in buckets[i]:
            res.append(num)
            if len(res) == k:
                return res
    return res
nums = [1,1,1,2,2,3]
k = 2
topKFrequent(nums,k)

[1, 2]

Heap - O(KlogN)

In [66]:
from collections import Counter
import heapq 

def topKFrequent(nums: list[int], k: int) -> list[int]:
    count = Counter(nums)
    
    #create a list of tuples (fre, num) for heapifying (we maintain heap by values)
    heap = [(fre, num) for (num, fre) in count.items() ]
    heapq._heapify_max(heap)
    print(heap)

    mlist = []
    for i in range(k):
        mlist.append(heapq._heappop_max(heap)[1]) #get the num (heap[1])
    return mlist
nums = [1,1,1,2,2,3]
k = 2
topKFrequent(nums, k)

[(3, 1), (2, 2), (1, 3)]


[1, 2]

In [65]:
#624. Maximum Distance in Arrays
def maxDistance(arrays: list[list[int]]) -> int:    
    #init so that first array will be 0 diff
    mmin = arrays[0][-1]
    mmax = arrays[0][0]
    res = 0
    for array in arrays:
        #update maxDistance 
        res = max(res, array[-1] - mmin, mmax - array[0])
        mmin = min(mmin, array[0])
        mmax = max(mmax, array[-1])    
    return res

arrs = [[-8,-7,-7,-5,1,1,3,4],[-2],[-10,-10,-7,0,1,3],[2]]
maxDistance(arrs)


14

[6. Zigzag Conversion](https://leetcode.com/problems/zigzag-conversion)


<img src="./assets/l6.jpeg" width="500px"/>

In [16]:
#For each character, append it to the the correct row
# using direction  up/down alternatingly, we can process row to know which row to append the current char to
def convert(s: str, numRows: int) -> str:
    if numRows == 1 or numRows >= len(s):
        return s
    
    rows = [""] * numRows
    row, direction = 0, 1  # Start at row 0 and move down

    for char in s:
        rows[row] += char  # Add character to current row
        if row == 0:  # If at the top, move down
            direction = 1
        elif row == numRows - 1:  # If at the bottom, move up
            direction = -1
        row += direction  # Update row index
    
    return "".join(rows)  # Concatenate all rows
s = "PAYPALISHIRING"
numRows = 4
convert(s, numRows)


'PINALSIGYAHRPI'

In [ ]:
''' 
charsInSection: diff = 2 * (numRows - 1)  # total chars in one section || big vertical step
for i in range(rows)
    while index < n:
        answer += s[index]  # Add the vertical character
        charsInBetween: diagonal_diff  = diff - 2 * i
        second_index = index + diagonal_diff  # diagonal index


'''
def convert(s: str, numRows: int) -> str:
    # Step 1: Handle the Edge Case
    if numRows == 1:
        return s  # If there's only one row, the result is the input string itself.

    # Step 2: Initialize Variables
    answer = ''  # Stores the final converted string
    n = len(s)  # Length of the input string
    diff = 2 * (numRows - 1)  # The "step size" for vertically aligned characters
    diagonal_diff = diff  # The step size for diagonal characters (calculated dynamically)
    second_index = 0  # Index for diagonal characters
    index = 0  # Current index in the string

    # Step 3: Iterate Through Each Row
    for i in range(numRows):
        index = i  # Start at the i-th character of the string for the current row
        while index < n:
            # Step 3a: Add the Vertical Character
            answer += s[index]  # Add the vertical character
            print(index)
            # Step 3b: Handle Diagonal Characters
            if i != 0 and i != numRows - 1:  # Only for rows between the first and last
                diagonal_diff = diff - 2 * i  # Calculate the step for the diagonal character
                second_index = index + diagonal_diff  # Compute the diagonal character's index
                if second_index < n:
                    answer += s[second_index]  # Add the diagonal character to the result string

            # Step 3c: Move to the Next Vertical Character
            index += diff  # Advance to the next vertical character in the current row
        print(answer)
    # Step 4: Return the Final Converted String
    return answer
    
s = "PAYPALISHIRING"
numRows = 4
convert(s, numRows)


In [ ]:
#38. Count and Say
class Solution:
    def countAndSay(self, n: int) -> str:
        # Step 1: Initialize the Starting String
        current_string = "1"  # The first string in the sequence is "1"

        # Step 2: Generate Subsequent Strings
        for _ in range(n - 1):  # Repeat the process n-1 times to generate the nth sequence
            next_string = ""  # Stores the next sequence in the iteration
            j = 0  # Pointer for the current group of digits
            k = 0  # Pointer for traversing and counting consecutive digits

            # Step 2a: Traverse and Count Consecutive Characters
            while j < len(current_string):
                while (
                    k < len(current_string)
                    and current_string[k] == current_string[j]  # Count consecutive digits
                ):
                    k += 1  # Increment k as long as characters match

                # Step 2b: Form the "Count and Say" Segment
                next_string += str(k - j) + current_string[j]  # Append count and the character
                j = k  # Move j to the next group of digits

            # Step 2c: Update the Current String for the Next Iteration
            current_string = next_string  # Prepare for the next iteration

        # Step 3: Return the nth Sequence
        return current_string


In [ ]:
#56. Merge Intervals
def merge(self, intervals: list[list[int]]) -> list[list[int]]:

    intervals.sort(key=lambda x: x[0])

    merged = []
    for interval in intervals:
        # if the list of merged intervals is empty or if the current
        # interval does not overlap with the previous, simply append it.
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            # otherwise, there is overlap, so we merge the current and previous
            # intervals.
            merged[-1][1] = max(merged[-1][1], interval[1])

    return merged

[48. Rotate Image](https://leetcode.com/problems/rotate-image/)

In [ ]:
class Solution:
    def rotate(self, matrix: list[list[int]]) -> None:
        self.transpose(matrix)
        self.reflect(matrix)

    def transpose(self, matrix):
        n = len(matrix)
        for i in range(n):
            for j in range(i + 1, n):
                matrix[j][i], matrix[i][j] = matrix[i][j], matrix[j][i]

    def reflect(self, matrix):
        n = len(matrix)
        for i in range(n):
            for j in range(n // 2):
                matrix[i][j], matrix[i][-j - 1] = matrix[i][-j - 1], matrix[i][j]
                
                
def rotate(matrix: list[list[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    #rotate outer layer first, then inner

    #l,r: same as top,down, for layer boundary
    l, r = 0, len(matrix) - 1
    while l < r: #move inward
        top, bottom = l, r

        for i in range(r-l): #use i variable to move along with direction each side, i is within range 0, r-l.
            topleft = matrix[top][l+i]
            matrix[top][l+i] = matrix[bottom-i][l]
            matrix[bottom-i][l] = matrix[bottom][r-i]
            matrix[bottom][r-i] = matrix[top+i][r]
            matrix[top+i][r] = topleft
        l+=1
        r-=1
    return matrix
            
                





        
        

[2491. Divide Players Into Teams of Equal Skill](https://leetcode.com/problems/divide-players-into-teams-of-equal-skill/description/?envType=company&envId=paypal&favoriteSlug=paypal-all)

In [ ]:
'''
Constraints:
skill.length is even.
1 <= skill[i] <= 1000
We can sort and use 2 pointers. But that's O(nlogn). Use frequency hashmap: O(n) time, 0(1) space bc contraints of frequency hashmap is 1000
'''

class Solution:
    def dividePlayers(self, skill: list[int]) -> int:
        n = len(skill)
        total_skill = sum(skill)
        skill_frequency = {}

        # Calculate total skill and skill frequency
        for s in skill:
            skill_frequency[s] = skill_frequency.get(s, 0) + 1

        # Check if total skill can be evenly distributed among teams
        if total_skill % (n // 2) != 0:
            return -1

        #Get target skill from total skill and #players
        target = total_skill // (n // 2)
        total = 0

        # Calculate total chemistry while verifying valid team formations
        for s in skill:
            partner_skill = target - s

            # Check if a valid partner exists
            if skill_frequency[partner_skill] == 0:
                return -1

            # Calculate and add chemistry of the current pair
            total += s * partner_skill
            skill_frequency[partner_skill] -= 1

        # Return half of total as each pair is counted twice
        return total // 2

In [ ]:
#1010. Pairs of Songs With Total Durations Divisible by 60
'''
(a+b) % 60=0 <=> Either: 1. a % 60=0 AND b%60=0; 2 (a % 60)+(b % 60)=60
Hashmap/array remainders with size 60 to record the frequencies of each remainder - as the range of remainders is [0,59].
Time: linear, space: constant
'''

from collections import defaultdict
def numPairsDivisibleBy60(time: list[int]) -> int:
    remainders = defaultdict(int)
    ret = 0
    for t in time:
        if t % 60 == 0: # check if a%60==0 && b%60==0
            ret += remainders[0]
        else: # check if a%60+b%60==60
            ret += remainders[60-t%60]
        remainders[t % 60] += 1 # remember to update the remainders
    return ret

In [ ]:
#54 Spirtal Matrix
def spiralOrder(matrix: list[list[int]]) -> list[int]:
    result = []
    rows, columns = len(matrix), len(matrix[0])
    up = left = 0
    right = columns - 1
    down = rows - 1

    while len(result) < rows * columns:
        # Traverse from left to right.
        for col in range(left, right + 1):
            result.append(matrix[up][col])

        # Traverse downwards.
        for row in range(up + 1, down + 1):
            result.append(matrix[row][right])

        # Make sure we are now on a different row.
        if up != down:
            # Traverse from right to left.
            for col in range(right - 1, left - 1, -1):
                result.append(matrix[down][col])

        # Make sure we are now on a different column.
        if left != right:
            # Traverse upwards.
            for row in range(down - 1, up, -1):
                result.append(matrix[row][left])

        left += 1
        right -= 1
        up += 1
        down -= 1

    return result

In [242]:
#1328. Break a Palindrome
class Solution:
    def breakPalindrome(self, palindrome: str) -> str:
        # Step 1: Handle Edge Case
        if len(palindrome) == 1:
            return ""  # A single-character palindrome cannot be made non-palindromic
        
        # Step 2: Convert String to List (Since Strings are Immutable)
        palindrome = list(palindrome)
        
        # Step 3: Try to Replace the First Non-'a' Character with 'a'
        for i in range(len(palindrome) // 2):  # Only iterate over the first half
            if palindrome[i] != 'a':  # Find the first non-'a' character
                palindrome[i] = 'a'  # Replace it with 'a' to break the palindrome
                return "".join(palindrome)  # Convert list back to string and return
        
        # Step 4: If All Characters are 'a', Change the Last Character to 'b'
        palindrome[-1] = 'b'
        
        # Step 5: Return the Modified String
        return "".join(palindrome)


In [ ]:
# 2554. Maximum Number of Integers to Choose From a Range I
class Solution:
    def maxCount(self, banned: list[int], n: int, maxSum: int) -> int:
        banned_set = set(banned)  # Use a set for quick lookups
        total_sum = 0  # Track the cumulative sum
        count = 0  # Track the count of valid numbers

        for i in range(1, n + 1):
            if i in banned_set:
                continue  # Skip banned numbers
            total_sum += i
            if total_sum > maxSum:
                break  # Stop if the sum exceeds maxSum
            count += 1

        return count

In [ ]:
''' * Idea is to find the frequencies/(number of times an index appear
* in the requests) and then keep the maximum number from input array
* in the max frequency index and so on.
*
* Example: input array [1,2,3,4,5] requests [[1,3],[0,1]]
* frequencyOfRequests =[1,2,1,1,0];
*
* Since we are try to get maximum sum we have to position max number in the input array in the index such that, index has maximum frequency.
* Now sort the input array and frequencyOfRequests so that we will have max number and max frequency in first indexes.
* After sorting:
* input array = [5,4,3,2,1] 
* frequencyOfRequests =[2,1,1,1,0];
* 
*  for i in input array { 
*   sum += inputArray[i] * frequencyOfRequests[i] 
*   since sum can be very large and can cause overflow int; sum = sum % (10^9 + 7);
*  };
*
* frequencyOfRequests can be found by iterating every request from start to end and mark the inputArray[index]++; in this case Time =                   * O(inputArray.size) * O(request.size) but with marking start index with +1 and (end + 1) index with -1 Time = (requests.size) * (1);
*
*  Time - r + n + nlog(n) + flog(f) + n -> O(nlog(n))
*  Space - O(n) for storing frequencies.'''
class Solution:
    def maxSumRangeQuery(self, nums: List[int], requests: List[List[int]]) -> int:
        n = len(nums)  # Length of the nums array
        count = [0] * (n + 1)  # Array to track the frequency of requests for each index

        # Step 1: Mark the start and end points of each request using the difference array technique
        for start, end in requests:
            count[start] += 1  # Increment at the start index
            count[end + 1] -= 1  # Decrement after the end index

        # Step 2: Compute the actual frequency of requests for each index using prefix sum
        for i in range(1, n):
            count[i] += count[i - 1]  # Add the previous frequency to the current index

        # Remove the extra element used for boundary handling
        count = count[:-1]  # Trim the last element to match the length of nums

        # Step 3: Sort both nums and count in descending order
        # This ensures the largest numbers are paired with the most frequently requested indices
        nums.sort(reverse=True)
        count.sort(reverse=True)

        # Step 4: Compute the maximum sum
        result = 0
        MOD = 10**9 + 7  # Modulus to prevent overflow
        for i in range(n):
            result += count[i] * nums[i]  # Multiply corresponding values and add to the result
            result %= MOD  # Apply modulus to keep the result manageable

        return result  # Return the maximum sum modulo 10^9 + 7


In [ ]:
class Solution:
    def subarraysDivByK(self, nums: list[int], k: int) -> int:
        count = 0
        n = len(nums)
        
        for i in range(n):
            curr_sum = 0
            for j in range(i, n):
                curr_sum += nums[j]
                if curr_sum % k == 0:
                    count += 1
        
        return count
    
    

[2768. Number of Black Blocks](https://leetcode.com/problems/number-of-black-blocks/)
Steps of evolved solutions:
1. Init a m.x grid with black cell = 1, count blocks around black cells, using a visited set to avoid counting on the same block twice. [Here](https://leetcode.com/problems/number-of-black-blocks/submissions/1831905480/)
2. Init a set of blackcells, count blocks around black cells, using a visited set to avoid counting on the same block twice. (save memory) [Here](https://leetcode.com/problems/number-of-black-blocks/submissions/1835598281/)

**Best**: Instead of: "Visit subgrid once, count how many blacks are inside" -> Do: "For every black cell, increment counts for all 2×2 blocks it belongs to"


## Two Pointers

**Same direction**.  
Pattern: 
- inplace array modify, one fast reader, one slow writter
- linked list position seeking.

In [ ]:
def move_zeros(nums: list[int]) -> None:
    first = 0
    #first is to write, second is to read, when second see a non-zero, first move and write.
    for second in range(len(nums)):
        if nums[second] != 0 :
            nums[first], nums[second] = nums[second], nums[first] #swap instead of nums[first]<->0 for second, to avoid case they're both at the same place.
            first+=1
    return nums


[167. Two Sum II - Input Array Is Sorted](https://leetcode.com/problems/two-sum-ii-input-array-is-sorted/)

In [ ]:
def twoSum(numbers: list[int], target: int) -> list[int]:
    #2 pointers: at start and at end. 
    # When sum(nums[i]+nums[j])> target -> decrement end p (decrease larger num in the sum)
    #sum(nums[i]+nums[j]) < target -> increase start p 
    #proof: 
    #Time: O(n), Space: O(1)

    pre, pos = 0, len(numbers)-1
    while pre < pos:
        sum = numbers[pre]+numbers[pos]
        if sum < target:
            pre += 1
        elif sum > target:
            pos -= 1
        else:
            return [pre+1, pos+1]
    return []

[15. 3Sum](https://leetcode.com/problems/3sum/)

In [61]:
def threeSum(nums: list[int]) -> list[list[int]]:
    nums.sort()
    res = []

    #i is pivot (leftest), l next to i, r is end of array
    for i in range(len(nums)):
        #no repeated triplet, nums[i-1] worked with everything on right of nums[i]
        if i > 0 and nums[i] == nums[i-1]:
            continue 
        #can't offset from positive pivot 
        if nums[i] > 0:
            break
        l, r = i+1, len(nums)-1
        while l < r:
            
            sum = nums[i]+nums[l]+nums[r]
            
            if sum == 0:
                res.append([nums[i],nums[l],nums[r]])
                #only need to update 1 pointer
                l+=1 
                #no repeated triplet
                while nums[l-1] == nums[l] and l<r :
                    l+=1
                
            elif sum > 0:
                r-=1
            elif sum < 0:
                l+=1
        
    return res
nums = [-1,0,1,2,-1,-4]
nums1= [-2,0,1,1,2]
display(threeSum(nums1))
display(threeSum(nums))

[[-2, 0, 2], [-2, 1, 1]]

[[-1, -1, 2], [-1, 0, 1]]

[1537. Get the Maximum Score](https://leetcode.com/problems/get-the-maximum-score/description/)

In [ ]:
class Solution:
    def maxSum(self, arr1: List[int], arr2: List[int]) -> int:
        #two pointers first, sec, and firstSum, secSum
        first, sec, firstSum, secSum = 0, 0, 0,0
        total = 0
        MOD = 10**9 + 7
        n, m = len(arr1), len(arr2)
        # gotta move first and second in a way that they will meet at "teleport gate" (common num)
        #by: move pointer a if its value smaller,then a will stop at gate, then b'll' move to gate soon after that.
        # update sectionSum after move pointer.
        while first < n or sec < m:
            # to teleport, or add the segment when the other reach len
            if first < n and sec < m and  arr1[first] == arr2[sec]:
            #total + higher segment + telegate
                total = (total+max(firstSum, secSum) + arr1[first] )
                #reset segment sum
                firstSum = 0
                secSum = 0
                #since we added gate above, move past gate and continue to avoid add gate to segment below
                sec+=1
                first+= 1
                continue
                
            #move first pointer when sec value is larger, sec reach len
            if sec==m or (first != n and arr1[first] < arr2[sec]): #must put sec==m first, so that when we enter the second or condition, sec was checked, and first also must be check in there 
                firstSum+=arr1[first]
                first += 1
            else:
                secSum += arr2[sec]
                sec += 1
        #add remaining segment that its pointer ends after the other to total
        total=(total+max(firstSum, secSum))
        
        return total% MOD

In [ ]:
#443. String Compression - inplace modify array 
def compress(chars: list[str]) -> int:
    slow, fast  = 0,1
    cur = 1
    while fast <= len(chars):
        #put fast == len(chars) first, if it's true, won't access chars[fast]
        if fast == len(chars) or chars[fast] != chars[fast-1]:
            if cur > 1:
                chars[slow] = chars[fast-1]
                count = str(cur)
                for c in count:
                    slow+=1
                    chars[slow] = c
                slow += 1
                cur = 1 #reset
            else:
                chars[slow] = chars[fast-1]
                slow +=1 
        else: 
            cur+=1
        fast+=1
    return len(chars[:slow])

chars = ["a","a","b","b","b","b","b","b","b","b","b","b","b","b","b","b","c","c","c","d"]
chars=["a","a","b","b","c","c","c"]


# chars=["a"]
compress(chars)

['a', '2', 'b', '2', 'c', '3']


7

In [ ]:
# 159. Longest Substring with At Most Two Distinct Characters, same as 904. Fruit Into Baskets
class Solution:
    def lengthOfLongestSubstringTwoDistinct(self, s: str) -> int:
        lastOccur = dict()  # {character: last occurence index of that character so far}
        l, r, longest = 0, 0, 0
        while r < len(s):
            lastOccur[s[r]] = r
            if len(lastOccur) > 2:
                # get the leftmost char(the char with smallest lastOccured index), set
                leftmostChar = min(lastOccur.values())
                longest = max(longest, r - l)
                l = leftmostChar + 1  # Left goes to right of that deleted char
                del lastOccur[s[leftmostChar]]
        return max(longest, r-l)
'''
"cacccabbb"
"caccccacabbbbbbbbb"
"caccccacaaaabbbbbbbbb"
'''

**Opposite direction**

[11. Container With Most Water](https://leetcode.com/problems/container-with-most-water) pretty much like 2 sum - opposite two pointers

In [ ]:
class Solution:
    def maxArea(self, height: List[int]) -> int:
        start, end = 0, len(height)-1
        largest = 0
        while start < end:
            area = (end - start)* min(height[end],height[start])
            if area > largest:
                largest = area
            if height[start] < height[end]: #choose to move the smaller
                start += 1
            else: end -= 1
        return largest


In [ ]:
def subarray_sum(arr: list[int], target: int) -> list[int]: #single subarr return [a,b]
    #inspired by twoSum
    #prefixSum = {prefixSum:index}; find complement on the way forming prefixSum
    prefixSum = {0:0} #{0:0} make it works for subarry starts at 0
    curSum = 0
    for i in range(len(arr)):
        curSum+= arr[i]
        complement = curSum - target
        if complement in prefixSum:
            return [prefixSum[complement], i+1]
        prefixSum[curSum] = i + 1 #we want prefixSum store sum up to i-1, to simplify code (i.e, return complementSum + 1, except for prefixSum[0])
        # then prefix[0] is 0 (i.e, if complement == 0 at i, return prefixSum[0]=0, not a i+1 like others)
    return []
#560. Subarray Sum Equals K
class Solution:
    def subarraySum(self, arr: List[int], target: int) -> int:
        # memory optimal solution - save map {curSum: #indices have that prefix}
        prefixSum = {0:1}
        curSum = 0
        count = 0
        for num in (arr):
            curSum += num
            if curSum - target in prefixSum:
                count+= prefixSum[curSum - target]
            prefixSum[curSum] = prefixSum.get(curSum, 0) + 1
        return count
#303. Range Sum Query - Immutable
class NumArray:
    def __init__(self, nums: List[int]):
        prefixSum = {0:0} #{index: sum upto index}
        for i, num in enumerate(nums):
            prefixSum[i+1] = prefixSum[i]+num
        self.prefixSum = prefixSum

    def sumRange(self, left: int, right: int) -> int:
        return self.prefixSum[right+1] - self.prefixSum[left]

## Stack

### Easy

[20. Valid Parentheses](https://leetcode.com/problems/valid-parentheses/)
1. Loop through the strings
2. If a open -> append to OpenStacks
3. Elif (close) & OF THE TOP of OpenStack: pop
4. Elif stack != 0 (still have non-open char -> illegal close char)

In [62]:
def isValid(s: str) -> bool:
    brack = {'{':'}', '[':']', '(':')'}
    stack = []
    for i in range(len(s)):
        if s[i] in brack: # if current visited char is OPEN
            stack.append(s[i])
        elif len(stack) != 0: #if current char is CLOSE
            if s[i] != brack[stack[-1]]: #but not match TOP OPEN => false
                return False
            stack.pop() #else pop
        # len(stack) == 0 + still have CLOSE char -> illegal close char
        elif len(stack) == 0: 
            return False
    if len(stack) != 0:
        return False
    return True

### Medium
Monotonic stack:
Efficient TRAVERSE and COMPARISON of elements in an ARRAY or sequence.
- Find the next nearest smaller || farthest greater
    - nearest smaller: use increasing stack (store largest so far, bc larger than that top is ignored, smaller than top is recorded)
- Monotonically Decreasing / Inc stack
- Lexicographical ordering

[155. Min Stack](https://leetcode.com/problems/min-stack/)

In [7]:
class MinStack:
    #Create a another minstack next to normal stack that stores upto current min val
    #minST always maintain the same size as stack
    def __init__(self):
        self.stack = []
        self.minSt = []

    def push(self, val: int) -> None:
        self.stack.append(val)
        #temp min
        # Only consider minSt[-1] if minSt not EMPTY
        self.minSt.append(min(val, self.minSt[-1] if self.minSt else val))


    def pop(self) -> None:
        #pop minSt always true. at any of its index, is always the stack's min up to that stack index: 
        # #st = [2, -1, 1, 0,-2]; min=[2,-1,-1,-1,-2]
        self.minSt.pop()
        self.stack.pop()

    def top(self) -> int:
        return self.stack[-1]

    def getMin(self) -> int:
        return self.minSt[-1]

minStack = MinStack()
minStack.push(-2)
minStack.push(0)
minStack.push(-3)
minStack.getMin()
minStack.pop()
minStack.top()
minStack.getMin()

print(f"Minimum element in the stack: {minStack.getMin()}")  

Minimum element in the stack: -2


[150. Evaluate Reverse Polish Notation](https://leetcode.com/problems/evaluate-reverse-polish-notation/)

This is postfix travel, utilize stack. Pop nums when encounter sign. Then push the result back to stack

In [64]:
import math
def evalRPN(tokens: list[str]) -> int:
    st = []
    res = 0
    for i in tokens:
        ch = i
        if ch == '+':
            res = st.pop() + st.pop()
        elif ch == '-':
            #later digit on top, pop first
            b,a = st.pop(), st.pop()
            res = a - b
        elif ch == '*':
            res = st.pop() * st.pop()
        elif ch == '/':
            #cast to int
            #e.g: -0.5 = 0, 8.9=8
            b,a = st.pop(), st.pop()
            res = int(a/b)
        else: #if num
            res = int(i)
        st.append(res)
    return st.pop()
tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"]
evalRPN(tokens)

6

[22. Generate Parentheses](https://leetcode.com/problems/generate-parentheses/)

[Generate Parentheses - Stack - Leetcode 22](https://www.youtube.com/watch?v=s9fokUqJ76A&t=1s)

<img src="assets/l22.png" width=800px />

In [10]:
def generateParenthesis(n: int) -> list[str]:
    out = []
    #comb = [] #each combination
    #Learned recursion TREE with # of o and c parentheses from 0 to n
    #add ( when open < n, add ) when close < open
    def backtrack(ocount, ccount, comb):
        #base case
        if ocount == n and ccount == n:
            out.append("".join(comb))
        
        if ocount < n:
            comb.append('(')
            backtrack(ocount+1, ccount, comb)
            comb.pop()
            
        if ccount < ocount: 
            comb.append(')')
            backtrack(ocount, ccount+1, comb)
            comb.pop()
    backtrack(0,0,[])
    return out
n=3
generateParenthesis(n)

['((()))', '(()())', '(())()', '()(())', '()()()']

[739. Daily Temperatures](https://leetcode.com/problems/daily-temperatures/)

Monotonic Decreasing Stack. O(n). Visit each elem once + Pop at most once -> O(2n)


In [40]:
def dailyTemperatures(temperatures: list[int]) -> list[int]:
    #Pattern reg: find nearest correlation -> stack.pop get the most recent val within the correlation.
    #Add temps to stack maintaining decreasingly monotonic, only (keep) pop when found the nearest warm (increase t)
    st = [] #index stack
    out = [0]*len(temperatures)
    
    for i,t in enumerate(temperatures):
        while st and t > temperatures[st[-1]]: #st[-1][0] = temp at top node
            index= st.pop()
            out[index] = i - index            
        st.append(i) #append all indices to st
    return out
dailyTemperatures([73,74,75,71,69,72,76,73])


[1, 1, 4, 2, 1, 1, 0, 0]

[962. Maximum Width Ramp](https://leetcode.com/problems/maximum-width-ramp/description/)

Max width ramp means: we care about the earliest smallest numbers.  
Approach:
- Use monotonic stack:  nums = [9,8,1,0,1,9,4,0,4,1] = > st=[9,8,1,0], what’s after doesn’t matter because the earliest smallest are here.  
- After forming the stack, go backwards nums bc we need latest end: endnum. 
    - Each endnum, keep popping stack from top while endnum > topnum.
    - why the rightmost end (j) has the right to keep popping the smallest (i)? When popped => already found the widest possible ramp for that i, and i can’t form a wider ramp .


In [36]:
def maxWidthRamp(nums: list[int]) -> int:
    st = [0] #(index)
    for i in range(1,len(nums)-1):
        if nums[i] < nums[st[-1]]: #decreasing order
            st.append(i)
    width = 0
    for i in range(len(nums) - 1, 0, -1):
        while st and nums[st[-1]] <= nums[i]:
            j = st.pop()
            if i - j > width: #get maxwidth at end i
                width = i - j  
    return width
nums = [9,8,10,7,6,5,1,0,1,9,4,0,4,1]
maxWidthRamp(nums)

9

In [4]:
#monotonic stack of all decreasing number that 
# not find its greater yet, and pop when found
def nextGreaterElements(nums: list[int]) -> list[int]:
    #find next greater elements for every number, return -1
    res = [-1]*len(nums)
    st = []
    for i, num in enumerate(nums):
        while st and nums[st[-1]] < num:
            #pop to get index and fill res
            res[st.pop()] = num     
        st.append(i)
    #second loop to search circularly
    if st:
        for i, num in enumerate(nums):
            while st and nums[st[-1]] < num:
                res[st.pop()] = num
    return res
nums = [1,2,3,4,3]
nextGreaterElements(nums)

[2, 3, 4, -1, 4]

<img src="assets/901stock.png" width="400px"/>

In [24]:
#901. Online Stock Span
# Use monotonic stack to skip past days with lower prices. Today's price is A.
# If A > stack.top B, pop B, add B span to A's.
# We will never visit B again bc B included in A’s span. Future price only deal with A and >A prices.
class StockSpanner:
    def __init__(self):
        self.stack = []

    def next(self, price: int) -> int:
        span = 1
        while self.stack and self.stack[-1][0] <= price:
            span+= self.stack.pop()[1] #get the span of the top (smallest)
        self.stack.append([price, span])
        return span
    
ops = ["StockSpanner","next","next","next","next","next","next","next"]
params = [[],[100],[80],[60],[70],[60],[75],[85]]
stockspanner = StockSpanner()
for op, param in zip(ops, params):
    if op == "next":
        result = stockspanner.next(param[0])  # Call next with the given price
        print("result",result)

result 1
result 1
result 1
result 2
result 1
result 4
result 6


<img src="assets/lc2866.jpg" width="400px">

In [ ]:
#2866. Beautiful Towers II 
#Note: some prevsmall is -1, we'll never deal with those prevsmall bc they're too small to be used in condition II
# No need to form arrays prev/nextSmall, top stack at each turn is i's prev/nextSmall.
def maximumSumOfHeights(maxHeights: list[int]) -> int:
    #use monotonic stack, each turn, top stack is always i's prevsmall, using it to cal leftCost
    n = len(maxHeights)
    st = [0]    
    left = [maxHeights[0]]
    #To find prevsmall: larger top must be popped -> increasing stack
    for i in range(1,n):
        #use stack to find i's prevSmall 
        while st and maxHeights[st[-1]] > maxHeights[i]: #if not i's smaller
            st.pop()
            
        #use prevSmall (stack top) to find leftsum
        #Case 1: re-use i-1 for i bc i-1 matches mountain pattern
        if maxHeights[i-1] <= maxHeights[i]:
            left.append(maxHeights[i] + left[i-1] )
        #Case 2: chop all larger past i to current i, ** stack top: i's prevSmall
        else:
            #leftsum = height[i]* distance of i - prevsmall + leftsum[prevsmall]
            if st: 
                left.append( maxHeights[i]*(i - st[-1]) + left[st[-1]] )
            else: # Case i is the smallest
                left.append( maxHeights[i]*(i+1) )
        #add i to stack
        st.append(i)
    st = [n-1]
    right = [0]*(n-1)+[maxHeights[-1]]
    #to find nextsmall: go from right to left, larger must be popped
    for i in range(n-2, -1, -1):
        # process stack to get top = i's nextSmall
        while st and maxHeights[st[-1]] > maxHeights[i]:
            st.pop()
        if maxHeights[i+1] <= maxHeights[i]:
            right[i] = maxHeights[i] + right[i+1]
        else:
            if st:
                right[i] = maxHeights[i] * (st[-1]-i) + right[st[-1]]
            else:
                right[i] = maxHeights[i] * (n - i)
        st.append(i)
    maxSum = 0
    for i in range(n):
        maxSum = max(maxSum, left[i] + right[i] - maxHeights[i])
    return maxSum
maximumSumOfHeights([5,9,9,8,1,7,8,3,4,3,1])
maximumSumOfHeights([6,5,3,9,2,7]
)

22

In [75]:
def maxChunksToSorted( arr: list[int]) -> int:
#for all i in sorted arrs: arr[i] = i. arr = [2,0,1,3,4], 
#when can we cut a range: when num is within its range: 4,2,3 is within 2-4, 5 at 5, 1,0 at 0,1. [1 0 4 2 3, 5]
#**if the max found upto i == i: we can cut max 2,0,1 == 2, cut here, 
        cuts = 0
        maxFound = 0
        for i, num in enumerate(arr):
            maxFound = max(maxFound, num)
            if maxFound == i:
                cuts+=1
        return cuts

maxChunksToSorted([1,2,0,3])

2

853. Car Fleet  

I. Sort by Starting Position - Because rightmost car's speed can't never be changed by prev cars whose speed must be higher.

II. Use a stack to store fleets's arrival time
    For each car:
- Check if car can catch up to top car before target by calculating car's time to reach target -> compare with top's time stored in stack
- If car can catch up, ignore, else: add car to stack as it's a new fleet. 

In [ ]:
def carFleet(target: int, position: list[int], speed: list[int]) -> int:
    st = []
    #sort by position Descending (front car firsts)
    cars = list(sorted(zip(position, speed), key=lambda x: x[0], reverse=True))
    
    for pos, speed in cars:
        arriveTime = (target - pos)/speed
        
        #top: front car,  #check if car can pass top's time b4 target
        if st and arriveTime <= st[-1]:
            #if pass: car will merge to top some time 
            #-> ignore car, and top remains the fleet's representative
                continue 
        #add new fleet if car can't pass top before target.           
        st.append(arriveTime)
    
    return len(st)
target = 12
position = [10,8,0,5,3]
speed = [2,4,1,1,3]
carFleet(target, position, speed)

3

In [ ]:
def mostCompetitive(nums: list[int], k: int) -> list[int]:
    #use increasing mono stack to optain sub-array that the first elements are smallest.
    n = len(nums)
    st = []
    for i in range(n):
        #when popping to get smaller first num, make sure the later nums enough to fill in popped num to form k-size subarr:
        #n-1 -i = #later nums to fill in, len(st)-1: st after popped
        while st and st[-1] > nums[i] and (n-1 -i + len(st)) >= k :
            st.pop()
        
        if len(st)<k:
            st.append(nums[i])

    return st
# nums = [2,4,3,3,5,4,9,6]
nums = [3,5,2,6]
k = 2
# k = 4
mostCompetitive(nums, k)

[2, 6]

### Hard

[84. Largest Rectangle in Histogram](https://leetcode.com/problems/largest-rectangle-in-histogram/)

In [65]:
"""
we need:  #figure left and right bound at once at any i
we do: as we go from left to right with a stack, the stack can help record:
- left bound of prev smaller: it is top of stack after pop bigger prev at i
    - right bound: at the time we pop bigger prev, bigger presv also knows that, the current i that pop them, is their right bound
=> we have: 
left bound of i: top of stack after pop bigger prev. store left bound with i in stack.
right bound of i: when i is popped by another bar j, j is right bound. cal area with stored left and right.
"""

def largestRectangleArea(heights: list[int]) -> int:
    st = [[0,-1]] # init st of (self, leftbound). first leftbound of 0 is -1.
    heights.append(0) # 1) Append a 0 to flush out all bars at the end **

    n = len(heights) 
    maxArea = 0
    for i in range(1,n):
        # print(st)
        while st and heights[st[-1][0]] >= heights[i]: # ** pop until found smaller
            #when we pop, calculate popped area, i is popped's rightbound
            idx, leftbound = st.pop()
            area = (i - 1 - leftbound) * heights[idx]
            # print(area, i, leftbound, heights[idx] )
            maxArea = max(maxArea, area)
        if st:
            # print(i, "append ", st[-1])
            st.append([i, st[-1][0]])
        else: #means no prev smaller -> i is smallest -> extend left to idx 0
            st.append([i, -1])
    
    return maxArea
heights = [2,1,5,6,2,3]
heights = [2,1,5,6,5,5,6,3]

largestRectangleArea(heights)

25

[2940. Find Building Where Alice and Bob Can Meet](https://leetcode.com/problems/find-building-where-alice-and-bob-can-meet/description/)

Approach and Time Complexity  
- Mono stack:
    - Loop in q, create `unsolveQ` at b, a < b, heights[a] > heights[b]: `unsolveQ[b].append([heights[a], queryIndex])`
    - Loop b backwards in heights:
        - Binary search in stack for unsolveQ[b]
            - Keep go to rightest (toppest) in stack where st[i] > st[unsolveQ[b].height[a]] => get smallest, nearest greater at stack
        - Pop st if top < b, those tops are unnecessary now, bc we st.append(n)
    - Time : **O(q⋅logn+n)**. Space : O(n+q) monostack + unsolveQ   

- Heap:
    - Loop in q, process and create exact same `unsolveQ`
    - Loop forward in heights, generate and use heap as we go:
        - At each height, look back (at heap) to see while height[heap[top]] < current height, answer it and pop.
        - Push all queries at height to heap.
    - Time : **O(q⋅logq+n)** . Space : O(n+q) minheap + unsolveQ  

In [167]:
'''
Already have process unresolve query at all maxIndex b in queries like previous stack approach
- With stack, we go right to left in height to look forward to see the nearest greater HEIGHT
- With heap, we go left to right in height to look backward to quickly find and resolve QUERY IN MINHEAP that has smaller heights (and ofc idx). Use heap, we can start quickly at the smallest.

Heap intuition approach:
Iterate throught building from l to r, want to find the building in the unsolved query that is smaller than current height.
=> use minheap to keep getting query with smallest height and (ofc smaller index) quickly.

'''
import heapq
def leftmostBuildingQueries(heights: list[int], queries: list[list[int]]) -> list[int]:  
    n = len(heights)
    q = len(queries)
    unsolveQ = [[] for _ in range(n)]
    res = [-1] * q
    
    #1. Process queries to have unsolveQ[b] = [heights[a], queryIndex], b > a, height(a > b)
    
    for i in range(q):
        a, b = queries[i][0], queries[i][1]
        if a > b:
            a, b = b, a
        if heights[a] < heights[b] or a==b:
            res[i] = b
        else:
            unsolveQ[b].append([heights[a], i])
    minheap = []   
    #2. From l to r in height, generate and use minheap as we go. At each height, quickly find qs with smaller height+index
    # ofc the higherIndex in heap's queries always smaller than current index, heap is processed up to current.)
    for i in range(n):
        #2.2, Solve all qs with smaller heights: chekc if heap[top] < height, if yes, record answer and pop (resolved) heap.
        while minheap and minheap[0][0] < heights[i]:
            res[minheap[0][1]] = i
            heapq.heappop(minheap)

        #2.1. Add queries at height to minheap (after find smaller height+index buildings at height.)
        for query in unsolveQ[i]:
            heapq.heappush(minheap, query)
    return res 
heights = [5,3,8,2,3,5,2,4,6,5,5,7]
queries = [(0,3), (5,6), (8,10), (0,6), (7,2), (6,1), (1,5), (0,7), (3,5)]
leftmostBuildingQueries(heights, queries)


[8, 8, 11, 8, -1, 7, 5, 8, 5]

In [72]:
#Find nearest greater on the right -> use mono stack + bin search
def leftmostBuildingQueries(heights: list[int], queries: list[list[int]]) -> list[int]:
    #1. make sure swap all queries q (a,b): a < b
    #resolve all easy queries height. if height(a>= b), unsolveQ[b].append([heights[a], queryindex])
   
    n = len(heights)
    q = len(queries)
    res = [-1]*q
    unsolveQ = [[] for _ in range(n)] # index at [b] = [height[a], queryIdx]
    for i in range(q):
        a, b= queries[i][0],  queries[i][1]
        if a > b: #swap
            a, b = b, a
        if heights[a] < heights[b] or a == b:
            res[i] = b
        else: #b: larger idx, smaller val
            unsolveQ[b].append([heights[a], i])
            
    st = [] #stack form [height b, idx b]
    #Loop thru position b (i), use stack to: 
    for i in range(n-1, -1, -1):
        # 1.right near greater of position b,...
        for height, qid in unsolveQ[i]:
            if st: #...found greater than a in stack using binary search
                pos = binSearch(height, st)
                res[qid] = pos
        # 2. Process stack: pop top < b(ofc num <  a bc b<a, top is useless now)
        while st and st[-1][0] <= heights[i]:
            st.pop()
        st.append([heights[i], i]) #height[i] for bin search comparison
    return res
      
# keep go right (means smallest - nearest/toppest) when mid > height in st 
def binSearch(height, st):
    left = 0
    right = len(st) - 1
    idx = -1
    while left<= right:
        mid = (left+right) // 2
        if st[mid][0] > height: 
            left = mid+1 #keep go rightest - smaller part
            idx = mid #mid always >= old idx bc mid always goes-right-to-idx once idx first got assigned by mid
        else:
            right = mid-1
    return st[idx][1] if idx != -1 else -1
# Time Complexity: O(q⋅logn+n)
# Space Complexity: O(n+q) monostack + newQuery    
    
heights = [5,3,8,2,6,1,4,6]
queries = [[0,7],[3,5],[5,2],[3,0],[1,6]]
heights = [5,3,8,2,3,5,2,4,6,5,5,7]
queries = [(0,3), (5,6), (8,10), (0,6), (7,2), (6,1), (1,5), (0,7), (3,5)]
leftmostBuildingQueries(heights, queries)

[[], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [[5, 0]], [], [], [[5, 1], [5, 3], [3, 5]], [[8, 4], [5, 7]], [], [], [[6, 2]], []]


[8, 8, 11, 8, -1, 7, 5, 8, 5]

```python
class Solution:
    def trap(self, height):
        # Step 1: Initialize Variables
        ans = 0  # Stores the total trapped water
        current = 0  # Pointer to traverse the height array
        st = []  # Stack to store indices of the heights
        
        # Step 2: Traversing the Heights
        while current < len(height):
            # Step 2a: Process When We Find a "Basin"
            while len(st) != 0 and height[current] > height[st[-1]]:
                # Step 2b: Identify the Water Trap
                top = st[-1]  # Get the index of the lowest point
                st.pop()  # Remove it from stack
                
                # Step 2c: Ensure a Left Boundary Exists
                if len(st) == 0:
                    break  # If the stack is empty, there's no left boundary, so no water can be trapped
                
                # Step 2d: Compute the Water Volume
                distance = current - st[-1] - 1  # Width of the trapped water region
                bounded_height = (
                    min(height[current], height[st[-1]]) - height[top]
                )  # Height of trapped water
                
                # Calculate trapped water for this segment
                ans += distance * bounded_height
            
            # Step 3: Push Current Height Index
            st.append(current)  # Store the current height's index in the stack
            current += 1  # Move to the next index
        
        return ans  # Return the total trapped water
```
1. **Monotonic Stack**:
   - Stores indices of heights in **increasing order**.
   - Helps in detecting **left and right boundaries** of water-trapping areas.

2. **Core Idea**:
   - When we find a **right boundary** (i.e., `height[current] > height[st[-1]]`), we pop elements from the stack to calculate trapped water.
   - The **distance** (`current - st[-1] - 1`) and **bounded height** (`min(height[current], height[st[-1]]) - height[top]`) determine the trapped water.

3. **Efficiency**:
   - **Time Complexity:** `O(n)` (Each element is pushed and popped at most once)
   - **Space Complexity:** `O(n)` (For storing indices in the stack)

your answer is a perfect template: adequate, concise, no yapping lengthy, very good, contain code with ggreat explanation steps comments, dsa, core idea and effiency. good! keep do this to 

In [8]:

def minSwaps(s: str) -> int:
    stack_size = 0
    for ch in s:
        # If character is opening bracket, increment the stack size.
        if ch == "[":
            stack_size += 1
        else:
            # If the character is closing bracket, and we have an opening bracket, decrease
            # the stack size.
            if stack_size > 0:
                stack_size -= 1
    return (stack_size + 1) // 2 # + 1 to handle odd case, where if stacksize is 
minSwaps('[]][')

1

## Binary search
**I. [Ultimate Guide](https://leetcode.com/discuss/post/786126/python-powerful-ultimate-binary-search-t-rwv8/)**
- Note that in FINDING **INSERTION** point in binary search, mostly need to 
    - **init r = len(arr) if using pref left**(insert to the left of found point, *foundpoint = len(arr) when x > all arrs*); 
    - or **init l = -1 if using pref right** (insert to the right of a found point, *foundpoint = -1 when x < all arrs*)
1) FIND MINIMUM t such that condition(t) == True (a.k.a. "first True", lower bound)
    ```python
    l, r = 0, x  # x must be a VALID candidate (or upper bound inclusive)
    while l < r:
        mid = l + (r - l) // 2   # mid prefers LEFT
        if condition(mid):       # mid is True -> could be the answer
            r = mid              # keep mid (search left side)
        else:                    # mid is False -> answer must be right
            l = mid + 1
    return l                     # l == first True
    ```
2) FIND MAXIMUM t such that condition(t) == True  (a.k.a. "last True in TTTFFF",  "floor")
- 2a: Similar template mid prefers LEFT above, [init r = x+1, return l-1](https://chatgpt.com/s/t_695ce2b9e81481918cb2efc7cd2b18ef)
    ```python
    l, r = 0, x + 1              # shift right fence to guaranteed-bad, to return l-1
    while l < r:
        mid = l + (r - l) // 2   # mid prefers LEFT
        if condition(mid):       # if mid is True (good) -> last True is >= mid
            l = mid + 1          # keep "good + 1" (move past mid)
        else:                    # else: mid is False (bad) -> first bad could be mid
            r = mid
    return l - 1                 # step back to last True, if there's no True, we'll get -1
    ```  
- 2b: MAX using mid prefers RIGHT
    ```python
    l, r = 0, x                  # here x must be VALID candidate max bound
    while l < r:
        mid = r - (r - l) // 2   # mid prefers RIGHT
        if condition(mid):       # mid is True -> could be answer, go right
            l = mid              # keep mid
        else:                    # mid is False -> must go left
            r = mid - 1
    return l                     # l == last True
    ```

**II. 3 main sections:**
- Pre-processing - Sort if collection is unsorted -> Binary Search -> Post-processing.  

**III. Paterns recognition:**
- Seach in sorted arrays. 
- Monotonicity: exists a function that map elements in left half to true, other half false.  
Find mid avoid overflow mid = l + ((r-l)//2)

[74. Search a 2D Matrix](https://leetcode.com/problems/search-a-2d-matrix/)

Bin search ROW (a range) -> bin search value in found ROW

Given matrix m*n: O(logm + logn) = O(log(m * n))

Turn into 1 1d array

In [13]:
def searchMatrix(matrix: list[list[int]], target: int) -> bool:
    ROW, COL = len(matrix), len(matrix[0])
    i,j = 0, ROW*COL-1 #start
    
    while i<=j:
        midIndex = (i+j)//2
        ro , co = midIndex//COL , midIndex % COL
        if matrix[ro][co] == target:
            return True
        if matrix[ro][co] < target:
            i = midIndex +1
        else:
            j = midIndex -1
    
    return False

matrix = [[1,3,5,7],[10,11,16,20],[23,30,34,60]]
target = 21
print(searchMatrix(matrix, target))

False


In [14]:
def searchMatrix(matrix: list[list[int]], target: int) -> bool:
    ROW, COL = len(matrix), len(matrix[0])
    top, bottom = 0, ROW - 1
    
    while top <= bottom:
        midRow = (bottom + top)//2
        if target > matrix[midRow][-1]:
            top = midRow + 1
        elif target < matrix[midRow][0]:
            bottom = midRow - 1
        else:
            break
    l, r = 0, COL -1 
    while l <= r:
        mid = (l+r)//2
        if target < matrix[midRow][mid]:
            r = mid - 1
        elif target > matrix[midRow][mid]:
            l = mid + 1
        else: 
            return True
    return False

Convert 2d -> 1d and do easy bin search

In [7]:
def searchMatrix(matrix: list[list[int]], target: int) -> bool:
    nums = [num for sublist in matrix for num in sublist]
    mid = len(nums)//2
    i,j= 0, len(nums)-1
    while i<=j:
        mid = (i+j)//2
        if target == nums[mid]:
            return True
        if target > nums[mid]:
            i = mid+1
        else:
            j = mid -1
    return False

matrix = [[1,3,5,7],[10,11,16,20],[23,30,34,60]]
target = 3
print(searchMatrix(matrix, target))

True


[875. Koko Eating Bananas](https://leetcode.com/problems/koko-eating-bananas/)

Lesson learned:
- If it seems way too complicated, start with worst brute force
- [0...k] search can always reduced to bin search

In [290]:
import math
def minEatingSpeed(piles: list[int], h: int) -> int:
    #h >= len(piles). Always. When h = len(piles), k = max(piles) (each hour, each pile)
    #In general: 1 <= k < = max (piles) for all h
    
    i, j = 1, max(piles)
    k = j

    while i <= j:
        mid = (i + j) // 2
        totalTime = 0
        
        for p in piles:
            totalTime += math.ceil(float(p) / mid)
            
        if totalTime <= h: #This means k is larger than should be, that it eats fater than given hours
            k = mid
            j = mid - 1
        else:
            i = mid + 1
    return k
            
piles = [3,6,7,11]
h = 8
minEatingSpeed(piles,h)


4

In [ ]:
def minEatingSpeed(piles: List[int], h: int) -> int:
    # find k, the smallest limit to eat whole piles the in the h hours
    # smallest pile <= k <= largest pile
    left, right = min(piles), max(piles)
    while left < right:
        mid = (left + right)//2
        #try to eat "mid" amount of bananas to see if it fits
        hours = 0
        for p in piles:
            hours += mid // p
        if hours > h: #eat more
            left = mid + 1
        else:
            right = mid - 1
        
            

[Find Minimum in Rotated Sorted Array](https://leetcode.com/problems/find-minimum-in-rotated-sorted-array)  
**Logn: Anything is a binary decision (going which way next) can be shrink to binary search range**  
Similar problem: 852. peak index in a mountain array  
- Don't try to store min/max value and make things complicated, just shrink down the search range until cannot.

In [ ]:
def findMin(nums: List[int]) -> int:
    # In rotated array, bigger half always in left, and first element > last element
    #BS below can narrow down to the lowest in sorted, so this part might not be needed
    # if nums[0] < nums[len(nums) - 1]: # so if array already sorted:
    #     return nums[0]

    left, right = 0, len(nums) - 1
    while left < right:
        mid = (left + right) // 2
        if nums[mid] < nums[right]:
            right = mid #guarantee right is always the minimum, what we need
        else:
            left = mid + 1 
    return nums[right]
def peakIndexInMountainArray(arr: List[int]) -> int:
        left, right = 0, len(arr) - 1
        while left < right:
            mid = (left + right)//2
            if arr[mid+1] > arr[mid]: # Compare to the next element only
                left = mid + 1 #guarantee left is always the maximum, what we need.
            else:
                right = mid 
        return left
# 69. Sqrt(x) - Example of Maximum
class Solution:
    def mySqrt(self, x: int) -> int:
        # find MAX element a, meets: a <= target => should do right pref
        # => r = mid-1, l = mid => mid must pref right to prevent inf loop
        l, r = 0, x
        while l < r:
            mid = r - (r-l)//2 # #10
            exp = mid*mid
            if exp > x: 
                r = mid - 1 #larger, must exclude mid
            else:
                l = mid
        return l
# -- transfer to pref left
class Solution:
    def mySqrt(self, x: int) -> int:
        #thinking process to figured out:
        #1. find minimal k satisfying condition k^2 > x (first true), then k - 1 to be the last false (max)
        #2.  mid pref left => l = mid + 1 (avoid inf loop), r = mid;
        # l = mid + 1 when exp(mid) <= x, mid is already satisfiable, so l=mid+1=satisfy+1
        # => satisfy = l - 1 at the end
        l, r = 0, x+1
        while l < r:
            mid = l + (r-l)//2 # #10
            exp = mid*mid
            if exp > x: 
                r = mid 
            else: #for all possible x, this will happen at least 1, because exp <= x always True for all x >= 0
                l = mid + 1
        return l - 1 # we'll never get -1, explained at else
    
class Solution:
    def getCommon(self, nums1: List[int], nums2: List[int]) -> int:
        #1st: Use 2 pointers to search 2 arrays: O(n+m)
        #2nd: sorted => monotonic=> can use binary search to optimize bruteforce nm -> n(logm). maximize benefits by log(larger)
        # 2nd way better than 1st only when m is much larger, worth the log.
        if len(nums1) > len(nums2):
            nums1, nums2 = nums2, nums1 #nums2 larger
        
        def binSearch(num, arr):
            l, r = 0, len(arr) - 1
            while l < r:
                mid = l+(r-l)//2
                if arr[mid] >= num:
                    r = mid
                else:
                    l = mid + 1 
            return arr[l] == num
        
        #binary search each num1 on nums2
        for num in nums:
            if binSearch(num, nums2):
                return num
        return -1
#162. Find Peak Element   
def findPeakElement(nums: list[int]) -> int:
        l = 0
        r = len(nums) - 1
        while l < r:
            mid = (l + r) // 2
            if nums[mid] > nums[mid + 1]:
                r = mid
            else:
                l = mid + 1
        return l
nums = [1,1,1,1,1,2,1,1,1,3,5,6,4]

[658. Find K Closest Elements](https://leetcode.com/problems/find-k-closest-elements/description/)

II. Most optimal: Use only binary search to find left bound
- Only need to find left bound, return [leftBoard:leftBoard+k]
- Since already had leftbound range: 0 -> (n-k) => can use binary search
- Mechanism to find leftbound:

e.g: [1,2,3,4,5,**8,9,12,18**,18,19,19,20], **x =13, k = 3**. 

- Examine candidate `mid`: compare `mid to x` vs `mid+k to x`(examine window size 4, k+1, [mid:mid+k+1]). *We can pick either of two, if one is closer, the other will never ever included in final res.*

- e.g: [mid] = 8 (window **[8,9,12]**) | [mid+k]=18 (window **[9,12,18]**)
    - If pick [mid], eliminate [mid+k] => move r = mid *(not move r=mid+k or mid+1, mid+2... if so the window will contain[mid+k]!)*
    - If pick [mid+k], eliminate [mid] => move l = mid+1 (exclude l)
- Lesson learned: when we see the dual condition in sorted arr:
    - |a - x| < |b - x|, or
    - |a - x| == |b - x| and a < b
> don't use abs, compare x-a vs b-x will naturally meet 2 conditions above:
```python
'''
special case *1, prove why without abs work: arr = [1,1,2,2,2,2,2,3,3],k = 3.
in duplicates, [mid] = 2, [mid+k]=2. 
- if x smaller than dup (x=1), (1 - arr[mid] <= arr[mid+k]-1) helps move towards left bc: -1 <= 1.
- if x smaller than dup (x=3), (3 - arr[mid] > arr[mid+k]-3) helps move towards right bc: 1 > -1.
'''
```

In [18]:
def findClosestElements(arr: list[int], k: int, x: int) -> list[int]:
    l, r = 0, len(arr)-k
    while l < r:
        mid = l+(r-l)//2
        #here, don't use abs() for subtraction, bc it will not reflect the case of duplicates *1
        if x - arr[mid] <= arr[mid+k]-x: #pick [mid], exlucde [mid:k], move right
            r = mid    
        else: l = mid + 1
    return arr[l:l+k]
arr=[1,1,2,2,2,2,2,3,3]
k=3
x=3
findClosestElements(arr,k,x)

[2, 3, 3]

I. Binary search and sliding window approach.  
Teach me about: 
- **Quickly understand the mechanism of bisect_left(arr,x): return pos >= x (x will be inserted in pos's left)**
    ```python
    arr=[1,2,3,4,5], x=3 => pos = 2
    arr=[1,2,4,5], x=3 => pos = 2
    arr=[1,2,6], x=3 => pos = 2
    arr=[1,2,6], x=7 => pos = init(r)
    ```
3 way to expand window from any window init (below)
- **Start with empty window to avoid bias**


In [ ]:
'''1. return `[left:right+1]` (left, right are exact indices), so init is `left, right = pos, pos - 1`
   so when expanding, you choose between `arr[left-1]` and `arr[right+1]`, then decrement/increment `left/right` accordingly.
'''
def first():
    # pos range: 0 -> len(arr)
    left, right = l, l - 1  # l (pos): insertion position of x, start with empty window [left:right+1]
    while right - left + 1 < k:
        if left == 0:
            right += 1
        elif right == len(arr) - 1:
            left -= 1
        else:
            # compare next-left (left-1) vs next-right (right+1)
            if x - arr[left - 1] <= arr[right + 1] - x:
                left -= 1
            else:
                right += 1
    return arr[left:right + 1]

'''2. return `[left:right]` (left exact, right excluded), so init is `left, right = pos, pos`
   so when expanding, you choose between `arr[left-1]` and `arr[right]`, then decrement/increment `left/right` accordingly.
'''
def sec():
    # pos range: 0 -> len(arr)
    left = right = l  # l (pos): insertion position of x, start with empty window [left:right)
    while right - left < k:
        if left == 0:
            right += 1
        elif right == len(arr):
            left -= 1
        else:
            # compare next-left (left-1) vs next-right (right)
            if x - arr[left - 1] <= arr[right] - x:
                left -= 1
            else:
                right += 1
    return arr[left:right]

'''3. window is `[left+1:right]` (left and right are “outside pointers”), init is `left, right = pos-1, pos` (pos range: 0 -> len(arr))
   here `left` points to the next candidate on the left, and `right` points to the next candidate on the right.
   so when expanding, you compare `arr[left]` vs `arr[right]`, then move `left/right`. final answer is `arr[left+1:right]`.
'''
def third():
    # pos range: 0 -> len(arr)
    left, right = l - 1, l  # candidates just outside the window
    while right - (left + 1) < k:  # current window is arr[left+1:right]
        if left < 0:               # no more left candidates
            right += 1
        elif right == len(arr):    # no more right candidates
            left -= 1
        else:
            # compare next-left (left) vs next-right (right)
            if x - arr[left] <= arr[right] - x:
                left -= 1
            else:
                right += 1
    return arr[left+1:right]

In [ ]:
def findClosestElements(arr: list[int], k: int, x: int) -> list[int]:
    l, r = 0, len(arr) 
    while l < r:
        mid = l+(r-l)//2
        if arr[mid] >= x:
            r = mid
        else:
            l = mid + 1
    #if right of window is exluded window fence, then pos range is 0->len(arr)
    left, right = l, l #l (pos): x insertion pos, start with empty window [left:right]
    while right - left < k:
        if left == 0:
            right += 1
        elif right == len(arr):
            left -= 1
        else:
            if x - arr[left-1] <= arr[right] - x:
                left -= 1
            else:
                right += 1
    return arr[left, right]

[AlgoMonster Newspaper](https://algo.monster/problems/newspapers_split) | [1011. Capacity To Ship Packages Within D Days](https://leetcode.com/problems/capacity-to-ship-packages-within-d-days/description/)
- Don't think of distribution problem to n workers, think of: Is there a time limit T, where if no worker exceeds that, they will finish in minimum. 
- With any given time limit, we can just use greedy simulation to quickly distribute time, and test if that time limit is distributable given array
- It deduces to binary search: what's the smallest YES time?  
```
Time:  10  11  12  13  14  15  16  17  18  19  20  ...  
Works: No  No  No  No  No  No  No  No  Yes Yes Yes ...  
```

In [ ]:
def shipWithinDays(self, weights: List[int], days: int) -> int:
    left, right = max(weights), sum(weights)
    def possibleCapacity(cap):
        daysNeeded = 1
        temp = 0
        for w in weights:
            if temp+w <= cap:
                temp += w
            else:
                temp = w
                daysNeeded+=1
            if daysNeeded > days:
                return False
        return True #daysNeeded <= days
    while left < right:
        mid = left+(right-left)//2
        if possibleCapacity(mid): #mid is a high enough capacity limit of the ship to ship within days
            right = mid
        else:
            left = mid+1
    return left

In [ ]:
#34. Find First and Last Position of Element in Sorted Array
# Do binary search 2 times, one to find min, second find max
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        def binary_search(nums, target, is_searching_left):
            left = 0
            right = len(nums) - 1
            idx = -1
            while left <= right:
                mid = (left + right) // 2
                if nums[mid] > target:
                    right = mid - 1
                elif nums[mid] < target:
                    left = mid + 1
                else:
                    idx = mid
                    if is_searching_left: #find leftmost
                        right = mid - 1
                    else: #find rightmost
                        left = mid + 1
            return idx
        
        left = binary_search(nums, target, True)
        right = binary_search(nums, target, False)
        
        return [left, right]
            

[729. My Calendar I](https://leetcode.com/problems/my-calendar-i/description/)

In [ ]:
# Find insertion point idx where calendar[idx][0] > start, to insert [start,end] to the left of idx. So, it's leftmost pref-left.
class MyCalendar:

    def __init__(self):
        self.calendar = []

    def book(self, startTime: int, endTime: int) -> bool:
        

[1146. Snapshot Array](https://leetcode.com/problems/snapshot-array/description/)

In [ ]:
# Use history array where history[i] is a list records [snapID, valAtSnapID] ith. Use binSearch on history[i], to find the most recent up to snapID time (rightmost)
class SnapshotArray:

    def __init__(self, length: int):
        self.history = [[[-1, 0]] for _ in range(length)]
        self.snapID = 0
    def set(self, index: int, val: int) -> None:
        #If you set the same index multiple times before calling snap, you can overwrite last instead of appending:
        # if self.history[index][-1][0] == self.snapID:
        #     self.history[index][-1][1] = val
        # else:
            self.history[index].append([self.snapID, val]) #main point
    def snap(self) -> int:
        self.snapID += 1
        return self.snap-1
    def get(self, index: int, snap_id: int) -> int:
        #find the most recent up to (right before or equal to) snapID time (rightmost)
        l, r = 0, len(self.history[index])-1
        while l < r:
            mid = r-(r-l)//2
            if self.history[index][mid][0] <= snap_id:
                l = mid
            else:
                r = mid - 1
        return self.history[index][l][1]

In [ ]:
#create candleIdx array, find the leftmost candleIdx bigger than qleft, rightmost smaller than qright.
def platesBetweenCandles(s, queries):
    candles = []
    for i in range(len(s)):
        if s[i] == '|': candles.append(i)

    res = []
    for qleft, qright in queries:
        left_pos, right_pos = -1, -1

        # 1. find index of first candle that comes after qleft (leftmost)
        left, right = 0, len(candles)-1
        while left < right:
            mid = left + (right-left) // 2
            if candles[mid] >= qleft:
                right = mid
            else:
                left = mid + 1
        left_pos = left
        # 2. find index of last candle that comes before qright (rightmost)
        left, right = 0, len(candles)-1
        while left < right:
            mid = right - (right-left) // 2
            if candles[mid] <= qright:
                left = mid
            else:
                right = mid - 1
        right_pos= left
        # result = range between two outermost candles - candle count in between
        if (left_pos != -1) & (right_pos!= -1) & (right_pos > left_pos):
            res.append((candles[right_pos] - candles[left_pos]-1) - (right_pos - left_pos-1))
        else:
            res.append(0)
    return res

In [ ]:
from bisect import bisect_left
#binary seach + greedy
class Solution:
    def lengthOfLIS(self, nums: list[int]) -> int:
        # Step 1: Initialize an Array to Track the LIS
        sub = []  # sub will store the smallest ending values for increasing subsequences
        
        # Step 2: Traverse the Array
        for num in nums:
            # Find the position to replace or extend
            i = bisect_left(sub, num)  # Binary search for the position

            # Step 2a: Extend the LIS if num is larger than all elements in sub
            if i == len(sub):
                sub.append(num)
            
            # Step 2b: Replace the smallest number greater than or equal to num
            else:
                sub[i] = num
        
        # Step 3: Return the Length of the LIS
        return len(sub)
#time: nlogn, space: n

In [ ]:
# DP represents the length of longest increasing subsequence ending at index i.
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        # Step 1: Initialize DP Array
        dp = [1] * len(nums)  # dp[i] stores the LIS ending at index i
        
        # Step 2: Populate DP Array
        for i in range(1, len(nums)):
            for j in range(i):  # Check all previous elements
                # If nums[i] can extend the subsequence ending at nums[j]
                if nums[i] > nums[j]:
                    dp[i] = max(dp[i], dp[j] + 1)  # Update dp[i] to include nums[i]

        # Step 3: Return the Maximum Length
        return max(dp)  # The LIS is the largest value in dp


In [ ]:
#528. Random Pick with Weight
import random

class Solution:
    def __init__(self, w: list[int]):
        """
        Step 1: Precompute Prefix Sums
        - Create a prefix sum array to store cumulative weights.
        - This allows us to efficiently pick an index based on weighted probability.
        """
        self.prefix_sums = []
        prefix_sum = 0

        for weight in w:
            prefix_sum += weight  # Keep adding to get cumulative sum
            self.prefix_sums.append(prefix_sum)  # Store in the list

        self.total_sum = prefix_sum  # Store the total sum for scaling random values

    def pickIndex(self) -> int:
        """
        Step 2: Generate a Weighted Random Index
        - Generate a random number in the range [0, total_sum).
        - Use binary search to efficiently find the corresponding index.
        """
        target = self.total_sum * random.random()  # Scale random float [0, 1) to [0, total_sum)
        
        # Step 3: Perform Binary Search to Locate the Target in Prefix Sums
        low, high = 0, len(self.prefix_sums)

        while low < high:
            mid = low + (high - low) // 2
            if target > self.prefix_sums[mid]:  # Move right if target is greater
                low = mid + 1
            else:  # Move left if target is within this prefix sum
                high = mid
        
        return low  # The index found corresponds to the weighted random pick


In [ ]:
#380. Insert Delete GetRandom O(1)
'''
Time: GetRandom O(1). Insert and Delete O(1) average, O(N) worst-case when the operation exceeds capacity of currently allocated array/hashmap and invokes space reallocation.
Space O(N)
'''

from random import random

class RandomizedSet:
    def __init__(self):
        """
        Step 1: Initialize Data Structures
        - `self.indices`: A dictionary to store the index of each element in `self.nums`.
        - `self.nums`: A list to store the elements in the set.
        """
        self.indices = {}  # Maps values to their index in the list
        self.nums = []  # Stores inserted elements

    def insert(self, val: int) -> bool:
        """
        Step 2: Insert a Value in O(1) Time
        - If `val` is already in the set, return False.
        - Otherwise:
          - Append `val` to `self.nums`.
          - Store its index in `self.indices`.
          - Return True (successful insertion).
        """
        if val not in self.indices:
            self.nums.append(val)
            self.indices[val] = len(self.nums) - 1  # Store index of val
            return True  # Successful insertion

        return False  # Value already exists

    def remove(self, val: int) -> bool:
        """
        Step 3: Remove a Value in O(1) Time
        - If `val` exists in the set:
          - Swap `val` with the last element in `self.nums` (to maintain O(1) deletion).
          - Update the index of the swapped element in `self.indices`.
          - Remove the last element from `self.nums` and delete `val` from `self.indices`.
          - Return True (successful removal).
        - Otherwise, return False.
        """
        if val in self.indices:
            # Get index of val and the last element
            index_to_remove = self.indices[val]
            last_element = self.nums[-1]

            # Swap val with last element in the list
            self.nums[index_to_remove] = last_element
            self.indices[last_element] = index_to_remove  # Update index of swapped element

            # Remove the last element
            self.nums.pop()
            del self.indices[val]  # Remove val from the dictionary

            return True  # Successful removal

        return False  # Value does not exist

    def getRandom(self) -> int:
        """
        Step 4: Get a Random Element in O(1) Time
        - Use `random()` to generate a random index in the range [0, len(self.nums) - 1].
        - Return the element at that index.
        """
        return self.nums[int(random() * len(self.nums))]  # Get a random index and return the value


## Sliding Window

l = 0;  
for r in len(arr):  

    determine when len(window) reaches max to reduce with while
    


### Easy

[121. Best Time to Buy and Sell Stock](https://leetcode.com/problems/best-time-to-buy-and-sell-stock/)

In [2]:
def maxProfit(prices: list[int]) -> int:
    buy = prices[0] #buy price
    mx = 0 #max profit
    for i in prices:
        if i < buy: #update best buy price
            buy = i
        elif i > buy and i - buy > mx: #update max profit
            mx = i - buy
    return mx

### Medium

[3. Longest Substring Without Repeating Characters](https://leetcode.com/problems/longest-substring-without-repeating-characters/)

In [3]:
def lengthOfLongestSubstring(s: str) -> int:
    l = 0 #left pointer
    longest = 0
    cset = set()
    for r in range(len(s)):
        #EXPAND TERMINATION: s[r] in cset
        while s[r] in cset: #cannot proceed with this string's start l, remove that.
            cset.remove(s[l])
            l+=1

        cset.add(s[r])
        longest = max(longest, r-l+1)
    return longest

[424. Longest Repeating Character Replacement](https://leetcode.com/problems/longest-repeating-character-replacement/)

In [25]:
def characterReplacement(s: str, k: int) -> int:
    l = 0
    longest = 0     #  length of longest valid window
    maxfreq = 0    # Keeps track of most freq character count in current window
    count = defaultdict(int)  # Dictionary to count characters in current window
    
    for r in range(len(s)):
        # Process window expansion
        count[s[r]] += 1
        
        # Update maxfreq if the newly added char is now more frequent
        maxfreq = max(maxfreq, count[s[r]])
        
        # WindowLength - maxfreq = numberCharsToBeReplace 
        while (r - l + 1) - maxfreq > k:
            #Process contraction: remove freq of char s[l]
            count[s[l]] -= 1  
            l += 1            
            
        longest = max(longest, r - l + 1)
    
    return longest

s = "ABCFBBBAACCCCC"
k = 4
characterReplacement(s, k)

9

[567. Permutation in String](https://leetcode.com/problems/permutation-in-string/)

Used 2 hashmap d1, d2 + sliding window

In [54]:
from collections import defaultdict
def checkInclusion(s1: str, s2: str) -> bool:
    d1, d2 = defaultdict(int), defaultdict(int)
    l = 0
    for i in s1:
        d1[i]+= 1
            
    for r in range(len(s2)):
        char = s2[r]
        if char not in d1:
            if d1 == d2:
                return True
            #reset d2
            d2.clear()
            continue
        
        if not d2: #set l = r (start new strs) after d2 reset to empty
            l = r

        d2[char]+= 1 #update count of char in d2
        #IMPORTANT: determine while when: d2[char] > d1[char]
        #Ensure ALL CHAR count in d2 <= d1.
        while d2[char] > d1[char]:
            d2[s2[l]] -=1
            l += 1
        if d1 == d2:
            return True
    return False
s1 = "adc"
s2 = "dcda"
checkInclusion(s1,s2)

True

Optimized sliding window with **match** count

Maintain a match variable, which stores the #chars (out of the 26 alphabets), THAT have the same occurence in **s1** and **current window in s2s2s2**. 

When we slide the window, 

- if the deduction of the last element and the addition of the new element leads to a new frequency match of any of the characters, we increment the match by 1. 
- Else if count of a char was the same earlier(prior to addition or removal) as in s1

    => NEW mismatch that was JUST USED TO be taken as prev **match**! -> decrement **match**.
     
- Else, do nothing, since **match** already up-to-date with all the matched and mismatched chars before.
After the shifting of the window, the count evaluates to 26, it means all the characters match in frequency totally. So, we return a True in that case immediately.

In [ ]:
def checkInclusion(s1: str, s2: str) -> bool:
    #This not check is essential because if move on, can't access s2[i] as s1[i] in next examine range(len(s1)):
    if len(s2) < len(s1):
        return False
    
    match = 0
    s1c,s2c = [0]*26, [0]*26
    
    #1. Get counts of s1,s2, up to len(s1):
    for i in range(len(s1)):
        #s1c records s1, s2c records s2
        s1c[ord(s1[i]) - ord('a')] += 1
        s2c[ord(s2[i]) - ord('a')] += 1
    
    #2. Init match of 26 chars:
    for i in range(26):
        if s1c[i] == s2c[i]:
            match += 1
    
    l = 0
    #3. Work on the rest
    for r in range(len(s1), len(s2)):
        if match == 26: return True
        
        #Move l + r, consider both moves
        indexR = ord(s2[r]) - ord('a') #right char
        s2c[indexR] +=1 #(I)
        #After (I) update        
        #Match ONLY INCRE when count of current char in 2 str matches <3
        #Match ONLY DECRE when what JUST PREVIOUS match => now mismatch, must be taken out of match
        if s1c[indexR] == s2c[indexR]:
            match += 1
        elif s1c[indexR] == s2c[indexR] - 1:
            match -=1 
        
        indexL = ord(s2[l]) - ord('a') #left char
        s2c[indexL] -=1
        if s1c[indexL] == s2c[indexL]:
            match += 1
        elif s1c[indexL] == s2c[indexL] + 1:
            match -=1 
        l+=1
    return match == 26

s1 = "adc"
s2 = "dcdba"
checkInclusion(s1,s2)


False

In [ ]:
#438. Find All Anagrams in a String (fixed window sliding)
from collections import Counter
class Solution:
    def findAnagrams(self, s: str, check: str) -> list[int]:
        n, m = len(s), len(check)
        countCheck = Counter(check)
        window = Counter(s[:m]) #counter of the first m characters in og

        res = []
        if window == countCheck: res.append(0)

        for r in range(m, n):
            
            window[s[r]] += 1
            window[s[r-m]] -= 1
            
            if window[s[r-m]] == 0:
                del window[s[r-m]]
                    
            if window == countCheck: #if they're anagrams
                res.append(r-m+1)
        return res

In [ ]:
#https://algo.monster/problems/subarray_sum_shortest
def subarray_sum_shortest(nums: list[int], target: int) -> int:
    l = 0
    shortest = -1
    tempSum=0
    for r in range(len(nums)):
        tempSum += nums[r]
        while tempSum-nums[l] >= target:
            tempSum -= nums[l]
            l+=1
        if tempSum >= target:
            shortest=min(shortest, r-l+1) if shortest != -1 else r-l+1
    return shortest
#2260. Minimum Consecutive Cards to Pick Up
class Solution:
    #better, using hashmap, not sliding window.
    def minimumCardPickup(self, cards: List[int]) -> int:
        last = {}
        ans = float('inf')
        for i, v in enumerate(cards):
            if v in last:
                ans = min(ans, i - last[v] + 1)
            last[v] = i
        return ans if ans != float('inf') else -1
    #my og sliding window
    def least_consecutive_cards_to_match(cards: list[int]) -> int:
        l = 0
        picked = set()
        minCards = float('inf')
        for r in range(len(cards)):
            while cards[r] in picked:
                picked.remove(cards[l])
                l+=1
            picked.add(cards[r])
            if cards[l-1] == cards[r] and l>0:
                minCards = min(minCards, r-l+2) #r-(l-1)+1 (bc previous l is match of r)
        return minCards if minCards != float('inf') else -1
        #watch longest exercises


In [ ]:
#1151. Minimum Swaps to Group All 1's Together
#there are k ones in total, then try to pick a window size k, and form all ones in the k-size window
# in window k size, there are z zeros -> try to turn all zeros to ones => find window ksize with min zero/max ones! 
class Solution:
    def minSwaps(self, nums: list[int]) -> int:
        count1 = nums.count(1) #fixed window size k needed to be explore
        maxOnes = nums[0:count1].count(1)
        curOnes = maxOnes
        for r in range(count1, len(nums)):
            curOnes += nums[r]
            curOnes -= nums[r-count1]
            maxOnes = max(maxOnes, curOnes)
        return count1 - maxOnes
#10010101|11010101
#00010|101011
arr= [0,1,1,1,0,0,1,1,0]
Solution.minSwaps(Solution, arr)



In [ ]:
#2134. Minimum Swaps to Group All 1's Together II (todo)

### Hard

[76. Minimum Window Substring](https://leetcode.com/problems/minimum-window-substring)

- **Key Variables**:
  - need, match: number of unique character: needed in t - has matched or exceeded so far
  - windCount, tCount: counter of window and t


In [ ]:
from collections import Counter, defaultdict
#shrink while the window remains valid, and update the answer during shrinking
# — because the best window is discovered by removing extras, not by adding the final required char.
def get_minimum_window(original: str, check: str) -> str:
    visited = defaultdict(int) #visited {char: count}; if use defaultdict, notice to not check eq with checkmap[i] for value = 0
    match = 0 #counts of chars match
    checkmap = Counter(check)
    res, resLen = [-1, -1], float('inf')
    l = 0
    
    for r in range(len(original)):
        visited[original[r]] += 1
        if visited[original[r]] == checkmap[original[r]] and visited[original[r]]!=0: 
            match += 1
        # shrink after found a candidate to be ready for new candidate
        while match == len(checkmap):
             # if match character and smaller len, update result, If same length, lexicographically smaller.
            if r-l+1 < resLen or (r-l+1 == resLen and original[l:r+1] < original[res[0]: res[1] + 1]):
                res = [l, r]
                resLen = r-l+1
            #shrink
            visited[original[l]] -= 1
            if visited[original[l]] < checkmap[original[l]]:
                match-=1
            l += 1
    return original[res[0]: res[1] + 1]

original = "cdbaeeebaecd"
check = "abc"
get_minimum_window(original, check)   

## Linked List

### <a id='initll'>Init Linked List for testing.</a>

In [202]:
from typing import Optional

# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
    def __repr__(self):
        s = "("+ str(self.val) + ") => "
        tm = self.next
        while tm:
            s += str(tm.val) + " -> "
            tm = tm.next
        return s

def print_list(head: Optional[ListNode]) -> None:
    while head:
        print(head.val, end=" -> ")
        head = head.next
    print("None")

# Helper function to create a linked list from a list of values
def create_list(values):
    if not values:
        return None
    head = ListNode(values[0])
    current = head
    for val in values[1:]:
        current.next = ListNode(val)
        current = current.next
    return head



In [5]:
# Test Case 1: Basic Test
head1 = create_list([1, 2, 3, 4, 5])
print("Original Linked List:")
print_list(head1)
# reversed_head1 = reverseList(head1)
# print("Reversed Linked List:")
# print_linked_list(reversed_head1)

Original Linked List:
1 -> 2 -> 3 -> 4 -> 5 -> None


### Easy
Run Init [LinkedList](#initll) before testing methods

#### [206. Reverse Linked List](https://leetcode.com/problems/reverse-linked-list/)

Reverse: prev is always the reversed list, not head, or temp

In [12]:

def reverseList(head: Optional[ListNode]) -> Optional[ListNode]:
    next = None #real next's next
    prev = None
    temp = head
    
    while temp:
        #save next
        next = temp.next 
        #point to prev
        temp.next = prev
        #update prev&temp
        prev = temp
        temp = next

    print(head)
    return prev #at the end, prev is head
reverseList(create_list([1,2,3,4]))

(1) => 


(4) => 3 -> 2 -> 1 -> 

#### [21. Merge Two Sorted Lists](https://leetcode.com/problems/merge-two-sorted-lists/)

##### Iterative

Create new list and gradually add next smaller nodes from two lists to the merged.

        Learned: temp = dummy
        
        temp.next = min(l1,l2); then temp = temp.next means temp helps adding things and act for dummy

In [ ]:
def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
    
        dummy = ListNode()
        temp = dummy
        
        while list1 and list2:
            if list1.val < list2.val:
                temp.next = list1
                list1 = list1.next
            else:
                temp.next = list2
                list2 = list2.next
            temp = temp.next
        
        temp.next = list1 or list2
        dummy = dummy.next
        return dummy

##### Recursive 
Without using another list

In [ ]:
def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:  
    def merging(list1, list2):
        
        #base:list1 or list2 completely examined
        if not list1:
            return list2
        if not list2:
            return list1
        
        #the smaller will be assigned its next
        #mergetwolist function acts like returning, providing the smaller node between two node args and keep moving on"
        if list1.val < list2.val:
            list1.next = merging(list1.next, list2)
            #providing the smaller results for the parent recursion
            return list1 
        else:
            list2.next = merging(list2.next, list1)
            return list2
    
    return merging(list1, list2)

#### [141. Linked List Cycle](https://leetcode.com/problems/linked-list-cycle/)


In [ ]:
def hasCycle(head: Optional[ListNode]) -> bool:
    #Guaranteed to meet when having a circle
    #Note: From meet point -> knot = head -> knot. 
    # ==> Find knot by move f to head, f and s incre 1 until meet
    f,s = head, head

    while f and f.next:
        f = f.next.next
        s = s.next
        if s == f:
            return True
    return False

### Medium

#### [143. Reorder List](https://leetcode.com/problems/reorder-list/)

**Approach 1** (Easy, slow): Create a arr, add each node to list in order. 

-> Reorder by indexing list 

In [3]:
def reorderList(head: Optional[ListNode]) -> None:
    """
    Do not return anything, modify head in-place instead.
    """
    if head.next == None or head.next.next == None:
        return head
        
    cur = head
    nodelist = list()
    while cur:
        nodelist.append(cur)
        cur = cur.next
    
    r = len(nodelist) - 1
    stop =len(nodelist) //2

    for l in range (stop):
        # todo: examine 3-4-5 case.   
        nodelist[l].next = nodelist[r]
        nodelist[r].next = nodelist[l+1]
        r-=1
    if len(nodelist) % 2 == 0:
        nodelist[r+1].next = None
    else:
        nodelist[r].next = None
    return head
    
head = [1,2,3,4]

head1 = create_list(head)
print_list(head1)

print(reorderList(head1))

1 -> 2 -> 3 -> 4 -> None
(1) => 4 -> 2 -> 3 -> 


**Approach 2**
1. Split into two halves. Using f,s pointers (s incre 1, f incre 2). When f no next -> s at middle
2. Reverse second half
3. Merge side by side 2 halves

In [4]:
import math
def reorderList(head: Optional[ListNode]) -> None:
    """
    Do not return anything, modify head in-place instead.
    """
    # firstnd middle
    s, f = head, head
    while f.next and f.next.next:
        s = s.next
        f = f.next.next
    
    #split
    second = s.next #saved secondcond half
    s.next = None #cut tail at mid
    #mid 's' still belongs to head -> cut tail of head
    
    #Reversecond
    prev = None
    next = None
    while second:
        next = second.next
        second.next = prev
        prev = second
        second = next
    
    second = prev #prev is reversecondd list
    
    #Merge 2 halves
    
    first = head #first half
    while first and second:
        fnext, snext = first.next, second.next
        first.next = second
        second.next = fnext
        first, second = fnext,snext
    return head #for testing
    
head = [1,2,3,4,5,6,7,8,9]

head1 = create_list(head)

print(reorderList(head1))

(1) => 9 -> 2 -> 8 -> 3 -> 7 -> 4 -> 6 -> 5 -> 


#### [19. Remove Nth Node From End of List](https://leetcode.com/problems/remove-nth-node-from-end-of-list/)

In [205]:
from typing import Optional
def removeNthFromEnd(head: Optional[ListNode], n: int) -> Optional[ListNode]:
    
    dummy = ListNode()
    #Dummy to remove head.
    dummy.next = head 
    l, r = dummy, head
    #r go n times
    for _ in range(n-1):
       r = r.next

    #push r to last
    while r.next: 
        r = r.next
        l = l.next
    l.next = l.next.next
    
    return dummy.next # return dummy.next instead of head to avoid situation remove head

head = [1,2]

head1 = create_list(head)
print(removeNthFromEnd(head1,2))
        
       

(2) => 


#### [287. Find the Duplicate Number](https://leetcode.com/problems/find-the-duplicate-number/description/)

In [ ]:
def findDuplicate(nums: list[int]) -> int:
    #floyd's hare and tortoise
    
    s,f = 0, 0
    #do while loop instead of while s!=f to run the first loop
    while True:
        s = nums[s]
        f = nums[nums[f]]
        if s == f:
            break

    #when s and f meet, set f to beginning
    f = 0
    #each takes 1 steps
    while s != f :
        s = nums[s]
        f = nums[f]
            
    return s

#### [2. Add Two Numbers](https://leetcode.com/problems/add-two-numbers/)

In [ ]:
def addTwoNumbers(l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
        res = ListNode()
        temp = res
        carr = 0 #carry
        
        #finsih loop while both l1 and l2 end
        while l1 or l2:
            
            if l1 and l2:
                ssum = l1.val + l2.val + carr
            elif l1:
                ssum = l1.val + carr 
            elif l2:
                ssum = l2.val + carr
            
            temp.val = ssum%10
            carr = ssum//10

            l1 = l1.next if l1 else None
            l2 = l2.next if l2 else None
            
            if l1 or l2:
                temp.next = ListNode()
                temp = temp.next
                
        if carr == 1:
            temp.next = ListNode()
            temp = temp.next
            temp.val = 1
        return res
l1 = create_list([9,9,9,9,9,9,9])
l2 = create_list([9,9,9,9])
addTwoNumbers(l1,l2)

(8) => 9 -> 9 -> 9 -> 0 -> 0 -> 0 -> 1 -> 

#### [146. LRU Cache](https://leetcode.com/problems/lru-cache/)

1. <b> <font color='yellow'>Lessons</font> </b>
Create nodes next to head and tail like left, right that points to head, tail. To refer head/tail, only used left.next or right.prev

-> To reduce complexity of code, reduce edge case where **head** and **tail** may not have **prev** and **next** like other nodes

<b> <font color='yellow'>Each time LRU change</font> </b>
Create 2 function: remove node, insert node to list
    

2. <b> <font color='yellow'>Approach</font> </b> LinkedList + HashMap for Put O(1) + Get O(1)
- array inefficient for changing the order of elements from most to least recently used

- To get any element O(1), we need to know the parent node of get node for order transition (getnode -> head, parent->get.next)
- To retrieve parent node O(1), node must point to **parent** 
                
            -> doule linkedlist

- hashmap for put only is ineffecient for remaining the order of most to LRU element, need to corporate **linkedlist**.




In [ ]:
class LRUCache:
#tailDummy -> nodes -> headDummy
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.head = Node(0,0) #most recent used
        self.tail = Node(0,0) #least recent used
        self.mdict = {}
        self.tail.next, self.head.prev = self.head, self.tail
        
        
    def remove(self,node):
        #all nodes have prev and next thanks to head and tail
        prev, next = node.prev, node.next
        prev.next = next
        next.prev = prev
        
    def insert(self,node):
        prev, next = self.head.prev, self.head
        prev.next = next.prev =node
        node.next, node.prev = next, prev
        
        
        
    def get(self, key: int) -> int:
        if key not in self.mdict:
            return -1
        node = self.mdict[key]
        #each time use the element, remove and reinsert
        self.remove(node)
        self.insert(node)
        return node.val
    
    def put(self, key: int, value: int) -> None:
        if key in self.mdict:
            self.remove(self.mdict[key])
            
        newNode = Node(key, value)
        #add to dict and list
        self.mdict[key] = newNode
        self.insert(newNode)
        
        #dismiss LRU
        if len(self.mdict) > self.capacity:
            lru = self.tail.next
            self.remove(lru)
            self.mdict.pop(lru.key)

    def __repr__(self):
        s = "("+ str(self.head.val) + ") => "
        tm = self.head.next
        while tm:
            s += str(tm.val) + " -> "
            tm = tm.next
        s+='\n'
        s+=str(self.mdict)
        return s
class Node():
    def __init__(self, key, val, next = None, prev=None):
        self.key = key
        self.val = val
        self.next = next
        self.prev = prev
    def __str__(self) -> str:
        return "(" +  str(self.key) + ": " + str(self.val) + ")"
    def __repr__(self) -> str:
        return "(" + str(self.key) + ": " + str(self.val) + ")"

get 1 -1
get 2 3
(0) => 
{2: (2: 3), 4: (4: 1)}


In [ ]:
#Use built-in
from collections import OrderedDict
class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.dic = OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.dic:
            return -1

        self.dic.move_to_end(key)
        return self.dic[key]

    def put(self, key: int, value: int) -> None:
        if key in self.dic:
            self.dic.move_to_end(key)

        self.dic[key] = value
        if len(self.dic) > self.capacity:
            self.dic.popitem(False)

# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

#### [138. Copy List with Random Pointer](https://leetcode.com/problems/copy-list-with-random-pointer/)


Approach: Go through 2 iterations
1. Create new deep copy nodes, map old nodes to new nodes in dict

2. Linked nodes' next and rands
- Link rands by mapping new nodes in dict by using relationships in old nodes




In [ ]:
#debug helpers
def create_linked_list(head_list):
    if not head_list:
        return None

    # Create nodes and store them in a dictionary with their indices
    nodes = {}
    for i, (val, _) in enumerate(head_list):
        nodes[i] = Node(val)

    # Connect nodes based on the second element of each sublist
    for i, (_, random_index) in enumerate(head_list):
        if random_index is not None:
            nodes[i].random = nodes[random_index]

    # Connect nodes based on their order in the original list
    for i in range(len(head_list) - 1):
        nodes[i].next = nodes[i + 1]

    return nodes[0]
class Node:
    def __init__(self, x: int, next: 'Node' = None, random: 'Node' = None):
        self.val = int(x)
        self.next = next
        self.random = random
    def __repr__(self):
        current = self
        s = ""
        while current:
            value = current.val
            random_node = current.random.val if current.random else None
            s+=f"({value}, {random_node}) -> "
            current = current.next
        s+="None"
        return s
    

In [ ]:
from typing import Optional

def copyRandomList(head: 'Optional[Node]') -> 'Optional[Node]':
        
        temp = head
        mdict = {None: None} #in case mdict[temp.next] or mdict[temp.random] null 
        
        #Map in dict {Key: Val} = {Original Node: New Node}
        while temp:
                mdict[temp] = Node(temp.val)
                temp = temp.next
        
        temp = head
        
        while temp:
                copiedNode = mdict[temp]
                #copiedNode is the copied verson of temp
                # mdict[temp.random] is the copied version of temp.random
                # => mdict[temp.random] is random version of newNode
                # So as "next"
                copiedNode.next = mdict[temp.next]
                #mdict[temp.random] is the deep copied node newNode.random
                copiedNode.random = mdict[temp.random]
                temp = temp.next
                
        return mdict[head]


head_list = [[7, None], [13, 0], [11, 4], [10, 2], [1, 0]]
head_node = create_linked_list(head_list)

copyRandomList(head_node)
        
        

(7, None) -> (13, 7) -> (11, 1) -> (10, 11) -> (1, 7) -> None

[1669. Merge In Between Linked Lists](https://leetcode.com/problems/merge-in-between-linked-lists)

In [ ]:
def mergeInBetween(list1: ListNode, a: int, b: int, list2: ListNode) -> ListNode:
    #1. get end list 2
    end2 = list2
    while end2.next:
        end2 = end2.next
    
    connect = list1
    #2. l1 go to connect start
    for i in range(a-1):
        connect = connect.next
    #3. remove: go to rest part
    rest1 = connect.next
    for _ in range(b-a+1):
        rest1 = rest1.next
        
    connect.next = list2 #connect to start of list2
    end2.next = rest1
    
    return list1

[237. Delete Node in a Linked List](https://leetcode.com/problems/delete-node-in-a-linked-list)

In [ ]:
class Solution:
    def deleteNode(self, node):
        # Overwrite data of next node on current node.
        node.val = node.next.val
        # Make current node point to next of next node.
        node.next = node.next.next

## Trees
A tree include trees.
1. Base case
2. Call function on left subtree (root.left)
3. Call function on right subtree (root.right)
4. Join function/return  
**Inorder**: left -> node -> right  
**Pre**: node -> left ->; **Post**: left->right->node

Trees height n:
- number of nodes: 2^n - 1; #leaf = 2^(n-1), #internal nodes: #leaf -1
Binary search tree: all left descendants < node < all right descendants

In [ ]:
from collections import deque
def inorderTraversal(root):
    stack = []
    result = []
    current = root

    while stack or current:
        # Reach the leftmost node
        while current:
            stack.append(current)
            current = current.left
        # Process the node
        current = stack.pop()
        result.append(current.val)
        # Visit the right subtree
        current = current.right

    return result
def preorderTraversal(root):
    if not root:
        return []
    
    stack = [root]
    result = []

    while stack:
        node = stack.pop()
        result.append(node.val)
        
        # Push right first so that left is processed first
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)

    return result
def postorderTraversal(root):
    if not root:
        return []
    
    stack = [root]
    result = []
    
    while stack:
        node = stack.pop()
        result.append(node.val)
        
        # Push left first so that right is processed first
        if node.left:
            stack.append(node.left)
        if node.right:
            stack.append(node.right)
    
    # Reverse the result because we need left -> right -> root
    return result[::-1]

def inorderDFS(root):
    if not root:
        return []
    return inorderDFS(root.left) + [root.val] + inorderDFS(root.right)
def BFS(root):
    q = deque([root])
    while q:
        for i in range(len(q)):
            node = q.popleft()
            print(node)
            if node.left: q.append(node.left)
            if node.right: q.append(node.right)


### Easy

#### Init Tree

In [236]:
from typing import Optional
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    def __repr__(self):
        result = "tree: {"
        queue = [self]

        while queue:
            current = queue.pop(0)

            if current:
                result += str(current.val) + " "

                # Enqueue left and right children
                queue.append(current.left)
                queue.append(current.right)
            else:
                result += "null "
        result += "}"
        return result.rstrip()
def to_tree(lst):
    if not lst:
        return None

    root = TreeNode(lst[0])
    nodes = [root]
    i = 1

    while nodes and i < len(lst):
        current = nodes.pop(0)

        if lst[i] is not None:
            current.left = TreeNode(lst[i])
            nodes.append(current.left)

        i += 1

        if i < len(lst) and lst[i] is not None:
            current.right = TreeNode(lst[i])
            nodes.append(current.right)

        i += 1

    return root
# root = [3,9,20,None,None,15,7]
# mtree = to_tree(root)
# mtree


#### [226. Invert Binary Tree](https://leetcode.com/problems/invert-binary-tree/)

In [ ]:
#DFS stack call
def invertTree(root: Optional[TreeNode]) -> Optional[TreeNode]:
    #if have no child
    if not root:
        return
    #perform first, call stack later
    temp = root.left
    root.left = root.right
    root.right = temp
    
    invertTree(root.left)
    invertTree(root.right)
    return root

#### [938. Range Sum of BST](https://leetcode.com/problems/range-sum-of-bst/description/?envType=daily-question&envId=2024-01-08)

- In BST, inorder traversal is sorted

Approach 1: Inorder Traversal  
Approach 2: Check each node
- If a node is larger than high -> don't go right
- If a node is smaller than low -> don't go left



In [ ]:
class Solution:
    def rangeSumBST(self, root: Optional[TreeNode], low: int, high: int) -> int:
        if not root:
            return 0
        if root.val >= low and root.val <= high:
            return root+ self.rangeSumBST(root.left,low, high)+ self.rangeSumBST(root.left,low, high)
        if root.val < low:
            return self.rangeSumBST(root.right)
        if root.val > high:
            return self.rangeSumBST(root.left)
root = to_tree([10,5,15,3,7,13,18,1,None,6])
low = 6 
high = 10
Solution.rangeSumBST(Solution, root, low, high)

23

#### [872. Leaf-Similar Trees](https://leetcode.com/problems/leaf-similar-trees/description/?envType=daily-question&envId=2024-01-09)
Get int[] of leaves's val of each tree and compare

In [ ]:
class Solution:
    def leafSimilar(self, root1: Optional[TreeNode], root2: Optional[TreeNode]) -> bool:
        if not root1 and not root2:
            return True
        def getleafdfs(node, res):
            if not node.left and not node.right:
                res.append(node.val)
                return res
            if node.left:
                res = getleafdfs(node.left, res)
            if node.right:
                res = getleafdfs(node.right, res)
            return res
        return  getleafdfs(root1, []) == getleafdfs(root2, [])
root1 = [3,5,1,6,2,9,8,None,None,7,4]
root2 = [3,5,1,6,7,4,2,None,None,None,None,None,None,9,8]

#### [104. Maximum Depth of Binary Tree](https://leetcode.com/problems/maximum-depth-of-binary-tree/) 
All solutions' complexity is O(n) <a id='maxdepth'></a>

**DFS Recursive**

In [ ]:
def maxDepth(root: Optional[TreeNode]) -> int:
    if not root:
        return 0
    return 1 + max(maxDepth(root.left), maxDepth(root.right))

**DFS Iterative**
1. Use a stack to record [[node, node.depth]]. Init with [root:1]
2. DFS because the added e to stack will go deep down (keep popping and processing child of child, because child of child added to top)
3. Pseudocode  
 While stack not empty:  
    pop node -> append {children : parent.level++}  
    maxlevel = max(maxlevel, node.level)  

In [72]:
from collections import OrderedDict
def maxDepth(root: Optional[TreeNode]) -> int:
    if not root:
        return 0
    maxDepth = 1
    stack = [[root, 1]]
    while stack:
        node, nodeDepth = stack.pop()
        if node:
            stack.append([node.left, nodeDepth+1])
            stack.append([node.right, nodeDepth+1])
            maxDepth = max(maxDepth, nodeDepth)
    return maxDepth

root = [3,9,20,None,None,15,7]
print(maxDepth(to_tree(root)))

3


**BFS (Iterative)**
1. Queue: deque([root]), level = 1
2. Each time popnode = deque.popleft:  Pop "node", add node's left, child if available. level++.
3. ...Until deque empty

In [ ]:
from collections import deque 
def maxDepth(root: Optional[TreeNode]) -> int:
    #edge case: empty root
    if not root:
        return 0 

    level = 0
    #init the queue of list [] starting with root
    q = deque([root])
    
    while q:
        for i in range(len(q)):
            popnode = q.popleft() #pop parents level and add children levels
            if popnode.left: q.append(popnode.left)
            if popnode.right: q.append(popnode.right)
        level+=1 #level increase
    return level

root = [3,9,20,None,None,15,7]
mtree = to_tree(root)
print(maxDepth(mtree))

tree: {3 9 20 null null 15 7 null null null null }
3


#### [543. Diameter of Binary Tree](https://leetcode.com/problems/diameter-of-binary-tree/)

***Brainstorm**: DFS gives us:  Each iter, we all can attain depth of two subtrees:
**Lchild.depth, Rchild.depth**

- p.depth = max(Lchild.depth,Rchild.depth) + 1
- p.diameter = Lchild.depth + Rchild.depth


Use DFS to find the max diameter of each node (subtree)
1. Go down to bottom left node. Start cal from that node.
2. Cal depth of l, r of that node.
3. Cal diamter of each node, compared to current maxdiameter

(DFS recursive): l, r = dfs(left), dfs(right)


In [ ]:
def diameterOfBinaryTree(root: Optional[TreeNode]) -> int:
     #global max variable
    maxDmter = 0
    
    #find height of any node AND continously update maxDiameter
    def dfs(root):
        if not root:
            return 0 #null height is -1
        
        # count this node's diameter -> compared to global max
        nonlocal maxDmter
        maxDmter = max(maxDmter, dfs(root.left) + dfs(root.right))
        
        return 1 + max(dfs(root.left), dfs(root.right)) #depth of node
    
    dfs(root)
    return maxDmter
root = [1,2,3,4,5,None,None]
mtree = to_tree(root)
print(diameterOfBinaryTree(mtree))

0


#### [110. Balanced Binary Tree](https://leetcode.com/problems/balanced-binary-tree/)

A height-balanced binary tree: depth of the two subtrees of every node never differs by more than one.

**Learned**: More optimal because keep returning -1 when finish, Look at **count** below

- Keep returning -1 will not bother this line return 1 + max(left, right) because:
- When -1 is assigned to eather left, right. The next step is keep returning -1 to the prev assigned parent (or left, right of grandparent again) and

and keep returning -1 instead of keep cal abs and 1+max.


In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        
        def dfs(root):
            if not root:
                return 0
            left = dfs(root.left)
            right = dfs(root.right)
            return -1 if left==-1 or right!=-1 or abs(left - right) > 1 else 1 + max(left, right)    
        return dfs(root) != -1
class Solution:
    #return (isBalance, height)
    def isBalanced(self, root: Optional[TreeNode]) -> bool:
        def dfs(node):
            if not node:
                return (True, 0)
            leftBalance, leftHeight = dfs(node.left)
            rightBalance, righttHeight = dfs(node.right)
            balance = leftBalance and rightBalance and abs(leftHeight-righttHeight)<=1
            return (balance, 1+ max(leftHeight, righttHeight))
        return dfs(root)[0]

In [ ]:
#DFS postorder stack iterative
class Solution:
    def isBalanced(self, root):
        stack = [(root, False)]  # (node, visited?) visited=False means "first time", True means "children done"
        depths = {None: 0}       # store computed heights; None has height 0

        while stack:
            node, visited = stack.pop()

            if not node:
                continue

            if not visited:
                # postorder simulation: push node back as "visited",
                # then push right/left to be processed first
                stack.append((node, True))
                stack.append((node.right, False))
                stack.append((node.left, False))
            else:
                # children already processed -> safe to compute height and balance
                left = depths.get(node.left, 0)
                right = depths.get(node.right, 0)

                if abs(left - right) > 1:
                    return False

                depths[node] = 1 + max(left, right)

        return True

root = [1,2,2,3,3,None,None,4,4, None, None, 5,5, None, None, 6,6, None, None, 7,7,None, None, 8,8]
mtree = to_tree(root)

#### [100. Same Tree](https://leetcode.com/problems/same-tree/)

**Top down approach**: Check from the root to l, r children.
- Only keep going down if not found unmatched yet.

In [ ]:
def isSameTree(p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
    if not p and not q: #equal if both NULLs
        return True
    if p and q and p.val == q.val: #only go down if equal
        
        return isSameTree(p.left, q.left) and isSameTree(p.right, q.right)
    
    #assign false to parent otherwise, stop going down here.
    return False 

True


**DFS approach**: Bottom up: check equal subtrees and pop up

In [ ]:
def isSameTree(p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
    
        if not p and not q:
            return True
        if p and q:
            l = isSameTree(p.left, q.left)
            if not l:
                return False
            r = isSameTree(p.right, q.right)
            if not r:
                return False
            return p.val == q.val
        
        return False


True


#### [572. Subtree of Another Tree](https://leetcode.com/problems/subtree-of-another-tree/description/)

Fastest: O(n+m+n) ~ Linear
1. DFS rec: get string traversal of sub and tree. check if sub in tree
Slower: O(mn)
2. DFS rec: While traversing dfs, check if subtree of each node in **root** is isSameTree as **subRoot**

In [ ]:
#Approach 1 DFS: O(n+m+n), less stack call 
def isSubtree(root: TreeNode, subRoot: TreeNode) -> bool:
    rootS = getStr(root)
    subS = getStr(subRoot)
    if subS in rootS: #(m+n)
        return True
    return False
 # add markers for null to create preorder complete tree -> unique tree with defined levels
def getStr(root):
    if not root:
        return " NUll "
    return str(root.val) + " "+  getStr(root.left) + getStr(root.right)


In [ ]:
#Approach 2: O(mn), more stack call, twice dfs
def isSubtree(root: TreeNode, subRoot: TreeNode) -> bool:
    if not subRoot:
        return True
    if not root:
        return False
    if isSameTree(root, subRoot):
        return True
    
    return isSubtree(root.left, subRoot) or isSubtree(root.right, subRoot)
    
    return False 

def isSameTree(q,p) -> bool:
    if not q and not p:
        return True
    
    if p and q and p.val == q.val:
        return isSameTree(p.left,q.left) and isSameTree(p.right,q.right) 
    
    return False #False otherwise    
root = [1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,2]
subRoot = [1,None,1,None,1,None,1,None,1,None,1,2]

print(isSubtree(to_tree(root), to_tree(subRoot)))

True


In [ ]:
#Approach 2: O(mn), DFS Iter
def isSubtree(root: TreeNode, subRoot: TreeNode) -> bool:
    stack = [root]
    while stack:
        popn = stack.pop()
        
        #code
        if isSameTree(popn, subRoot): return True
        
        if popn: 
            stack.append(popn.right)
            stack.append(popn.left)
        
    return False

def isSameTree(q,p) -> bool:
    if not q and not p:
        return True
    
    if p and q and p.val == q.val:
        return isSameTree(p.left,q.left) and isSameTree(p.right,q.right) 
    
    return False #False otherwise    
root = [1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,None,1,2]
subRoot = [1,None,1,None,1,None,1,None,1,None,1,2]

print(isSubtree(to_tree(root), to_tree(subRoot)))

True


### Medium

#### [235. Lowest Common Ancestor of a Binary Search Tree](https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-search-tree/)

In [ ]:
#since we traverse FROM ROOT, (smaller <= node.val and node.val <= greater) is enough to make node the lowest parent.
#DFS REC
def lowestCommonAncestor(root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
    greater = max( p.val, q.val)
    smaller = min(q.val, p.val) 
    def dfsInRange(root): 
        if smaller <= root.val and root.val <= greater:
            return root
        elif root.val > greater:
            return dfsInRange(root.left) 
        elif root.val < smaller:
            return dfsInRange(root.right) 
    return dfsInRange(root)
#Regular ITER
def lowestCommonAncestor(root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
    greater = max( p.val, q.val)
    smaller = min(q.val, p.val) 
    cur = root
    while cur:
        if smaller <= cur.val and cur.val <= greater:
            return cur
        elif cur.val > greater:
            cur = cur.left
        elif cur.val < smaller:
            cur = cur.right 
    return cur

#### [236. Lowest Common Ancestor of a Binary Tree](https://leetcode.com/problems/lowest-common-ancestor-of-a-binary-tree)

In [ ]:
def lowestCommonAncestor(root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:
        if not root or root == p or root == q:
            return root

        left = lowestCommonAncestor(root.left, p, q)
        right = lowestCommonAncestor(root.right, p, q)

        if left and right:
            return root
        return left if left else right

root = [3,5,1,6,2,0,8,None,None,7,4], 
p = 5, q = 1
lowestCommonAncestor(to_tree(root), TreeNode(p), TreeNode(q) )

In [ ]:
#iterative using stack
def lowestCommonAncestor(root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:

    # Stack for tree traversal
    stack = [root]

    # Dictionary for parent pointers, childKey : arentNode
    parent = {root: None}

    # Iterate until we find both the nodes p and q
    while p not in parent or q not in parent:

        node = stack.pop()

        # While traversing the tree, keep saving the parent pointers.
        if node.left:
            parent[node.left] = node
            stack.append(node.left)
        if node.right:
            parent[node.right] = node
            stack.append(node.right)

    # Ancestors set() for node p.
    ancestors = set()

    # Process all ancestors for node p using parent pointers.
    while p:
        ancestors.add(p)
        p = parent[p]

    # The first ancestor of q which appears in
    # p's ancestor set() is their lowest common ancestor.
    while q not in ancestors:
        q = parent[q]
    return q

#### [102. Binary Tree Level Order Traversal](https://leetcode.com/problems/binary-tree-level-order-traversal/)

In [ ]:
from collections import deque
#BFS
def levelOrder(root: Optional[TreeNode]) -> list[list[int]]:

    if root:
        queue = deque([root])
    else: return []
    res = []
    while queue:
        temp = [] #new list each level
        #for each node in the current level
        for i in range(len(queue)):
            node = queue.popleft()  
            temp.append(node.val)
            if node.left:
                queue.append(node.left)
            if node.right: 
                queue.append(node.right)
        #after each level
        res.append(temp)
    return res
root = to_tree([3,9,20,None,None,15,7])
print(levelOrder(root))
    

[[3], [9, 20], [15, 7]]


#### [199. Binary Tree Right Side View](https://leetcode.com/problems/binary-tree-right-side-view/)

In [ ]:
from collections import deque
def rightSideView(root: Optional[TreeNode]) -> list[int]:
        if not root:
            return []

        res = [root.val]
        queue = deque([root])
        while queue:
            for i in range(len(queue)):
                popnode = queue.popleft() #the rightest node in the queue
                if popnode.right:
                    queue.append(popnode.right)
                if popnode.left:
                    queue.append(popnode.left)

            if queue:
                #On each level, append the rightest (first) node in the queue.
          
                res.append(queue[0].val)
        return res
root = to_tree([1,2,3,None,5,None,4])
print(rightSideView(root))

[1, 3, 4]


#### [1448. Count Good Nodes in Binary Tree](https://leetcode.com/problems/count-good-nodes-in-binary-tree/)

DFS Rec approach
1. Each DFS, with record the current max of the traverse path of the node. Compared node with its max to incre **good**
2. Curmax of both left and right subtree is curmax of node.

In [ ]:
def goodNodes(root: TreeNode) -> int: 
    good = 0    
    def dfsMax(node, curmax):
        if not node:
            return 0
        
        if node.val >= curmax:
            curmax = node.val
            nonlocal good
            good+=1
        
        #curmax of node feed to left, right children
        dfsMax(node.left, curmax)
        dfsMax(node.right, curmax)
    
    dfsMax(root,root.val)
    
    return good 
     
root = to_tree([3,1,4,3,None,1,5] )
goodNodes(root)

4

In [ ]:
def goodNodes(root: TreeNode) -> int: 
    #a dfs that have curNode and max of the traverse path
    def dfs(node, mmax):
        if not node:
            return 0
        goodCounts = 0
        if node.val >= mmax:
            goodCounts = 1
            mmax = node.val   
        goodCounts += dfs(node.left, mmax)
        goodCounts += dfs(node.right,mmax)
        
        return goodCounts
    return dfs(root, root.val)
        
        

#### [98. Validate Binary Search Tree](https://leetcode.com/problems/validate-binary-search-tree/)
1. Recursive dfs: Idea of compare node's subtrees with itself
2. Pass the range (min, max) to each node where:
- For node.left, min is bounded by node's min, max = node
- For node.right, max is bounded by node's max, min = node

In [ ]:
def isValidBST(self, root: Optional[TreeNode]) -> bool:
    def dfs(node, nmin, nmax) -> bool:
        if not node:
            return True #if could reach base case eventually, return True and exit.
        if not (nmin < node.val < nmax):
            return False
        #Pass node's min and node's max to check children after check self.
        return dfs(node.left, nmin, node.val) and dfs(node.right, node.max, nmax)
    return dfs(root, float("-inf"), float("inf"))

#Iter stack version
def iterative(root):
    if not root: return True
    stack = [(root, -float('inf'), float('inf'))]
    while len(stack):
        node, nmin, nmax = stack.pop()
        if not (nmin < node.val < nmax): return False
        if node.left: stack.append((node.left, nmin, node.val))
        if node.right: stack.append((node.right, node.val, nmax))
    return True

#### [230. Kth Smallest Element in a BST](https://leetcode.com/problems/kth-smallest-element-in-a-bst/)

The smallest -> largest in BST is inOrder traversal: Left - self - right

**Recursive thinking:**
- We go left, self, right where we only perform at "self". Left, and right passed to is just travelling

In [ ]:
def kthSmallest(root: Optional[TreeNode], k: int) -> int:
    count = 0
    res = 0
    def dfsInOrder(node):
        if not node:
            return
        nonlocal count, res
        dfsInOrder(node.left) 
        
        #perform code
        if count == -1: #help exit faster
            return
        count+=1
        if count == k:
            res = node.val
            count = -1
            return 

        dfsInOrder(node.right)
        
    dfsInOrder(root) 
    return res

**Iter thinking**: cur = root will be the traveller.
Use 

    while cur:
            stack.append(cur) #all e in stack is not null
            cur = cur.left
            
In the begining of each loop to get the leftest of a node.

In [ ]:
def kthSmallest(root: Optional[TreeNode], k: int) -> int:
    count = 0
    stack = [root]
    cur = root
    while stack or cur: #incase stack empty but cur right is working
        #go all the way to bottom left before performing code
        while cur:
            stack.append(cur) #all e in stack is not null
            cur = cur.left
        #after while loop, cur is always None to the left.
        cur = stack.pop() 
        #perform at node
        count+=1
        if count == k:
            return cur.val
        #go to rights
        cur = cur.right

#### [105. Construct Binary Tree from Preorder and Inorder Traversal](https://leetcode.com/problems/construct-binary-tree-from-preorder-and-inorder-traversal/)

True Premises about arrays:
- Preorder: Top is root -> left subtree -> right subtree
- Inorder: left subtree -> root(midIndex) -> right subtree
Recursion DFS approach:
- Get curr node and its inorder index: root = preorder[0], midI = inorder.index(preorder[0])
- Assign left, right subtree with:
    - preorder_left = preorder[1:mid+1], preorder_right = preorder[mid+1:], 
    - inorder_left = inorder[:midI], inorder_right = inorder[mid+1:] (exclude midI)
- Return curr node


In [ ]:
class Solution:
    def buildTree(self, preorder: list[int], inorder: list[int]) -> Optional[TreeNode]:
        if not preorder or not inorder:
            return None
        root = TreeNode(preorder[0])
        midI = inorder.index(preorder[0])
        #DFS: Go to down leftest
        root.left = self.buildTree(preorder[1:midI+1], inorder[:midI])
        root.right = self.buildTree(preorder[midI+1:], inorder[midI+1:])
        return root

#### [114. Flatten Binary Tree to Linked List](https://leetcode.com/problems/flatten-binary-tree-to-linked-list/description/?envType=company&envId=paypal&favoriteSlug=paypal-all)

In [ ]:
#Recursive
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution(object):
    def flatten(self, root):
        """
        :type root: Optional[TreeNode]
        :rtype: None Do not return anything, modify root in-place instead.
        """
        self.flatTail(root)
    # Flatten tree + Return RIGHTMOST TAIL
    def flatTail(self,node):
        if not node:
            return None
        if not node.left and not node.right: 
            return node
        
        leftTail = self.flatTail(node.left)
        righTail = self.flatTail(node.right)
        
        if leftTail:
            leftTail.right = node.right
            node.right = node.left
            node.left = None
        return righTail if righTail else leftTail
    

In [ ]:
#Iterative using stack
from collections import deque
def flatten(root: Optional[TreeNode]) -> None:

    # Handle the null scenario
    if not root:
        return None

    START, END = 1, 2

    tailNode = None
    stack = deque([(root, START)])

    while stack:
        currentNode, recursionState = stack.pop()
        # We reached a leaf node. Record this as a tail
        # node and move on.
        if not currentNode.left and not currentNode.right:
            tailNode = currentNode
            continue

        # If the node is in the START state, it means we still
        # haven't processed it's left child yet.
        if recursionState == START:

            # If the current node has a left child, we add it
            # to the stack AFTER adding the current node again
            # to the stack with the END recursion state.
            if currentNode.left:
                stack.append((currentNode, END))
                stack.append((currentNode.left, START))
            elif currentNode.right:
                # In case the current node didn't have a left child
                # we will add it's right child
                stack.append((currentNode.right, START))
        else:
            # If the current node is in the END recursion state,
            # that means we did process one of it's children. Left
            # if it existed, else right.
            rightNode = currentNode.right

            # If there was a left child, there must have been a leaf
            # node and so, we would have set the tailNode
            if tailNode:
                # Establish the proper connections.
                tailNode.right = currentNode.right
                currentNode.right = currentNode.left
                currentNode.left = None
                rightNode = tailNode.right

            if rightNode:
                stack.append((rightNode, START))

#### [863. All Nodes Distance K in Binary Tree](https://leetcode.com/problems/all-nodes-distance-k-in-binary-tree)

In [ ]:
def distanceK(root: TreeNode, target: TreeNode, k: int) -> List[int]:
    # Recursively add a parent pointer to each node.
    def add_parent(cur, parent):
        if cur:
            cur.parent = parent
            add_parent(cur.left, cur)
            add_parent(cur.right, cur)

    add_parent(root, None)

    answer = []
    visited = set()

    def dfs(cur, distance):
        if not cur or cur in visited:
            return
        visited.add(cur)
        if distance == 0:
            answer.append(cur.val)
            return
        dfs(cur.parent, distance - 1) 
        dfs(cur.left, distance - 1)
        dfs(cur.right, distance - 1)

    dfs(target, k)

    return answer

#### [2385. Amount of Time for Binary Tree to Be Infected](https://leetcode.com/problems/amount-of-time-for-binary-tree-to-be-infected/)

In [ ]:
#turn to graph then bfs
def amountOfTime(root: Optional[TreeNode], start: int) -> int:
    def dfs(node):
        if node is None:
            return
        if node.left:
            graph[node.val].append(node.left.val)
            graph[node.left.val].append(node.val)
        if node.right:
            graph[node.val].append(node.right.val)
            graph[node.right.val].append(node.val)
        dfs(node.left)
        dfs(node.right)

    graph = defaultdict(list)
    dfs(root)
    visited = set()
    queue = deque([start])
    time = -1
    while queue:
        time += 1
        for _ in range(len(queue)):
            current_node = queue.popleft()
            visited.add(current_node)
            for neighbor in graph[current_node]:
                if neighbor not in visited:
                    queue.append(neighbor)
    return time

In [239]:
#dfs, maxlength is minutes. infected subtree got negative value and propagate negative to parent

def amountOfTime(root: Optional[TreeNode], start: int) -> int:
    res = 0  # Stores the maximum time required for infection to spread
    def DFS(node, start):
        nonlocal res
        if node is None:
            return 0  # Base case: If node is null, return depth 0
        
        # Recursively get the depth of left and right subtrees
        leftDepth = DFS(node.left, start)
        rightDepth = DFS(node.right, start)
        
        if node.val == start:
            # If current node is the infection source, update result with max subtree depth
            res = max(leftDepth, rightDepth)
            return -1  # Mark infection source to track spread
        
        elif leftDepth >= 0 and rightDepth >= 0:
            # If neither subtree contains the infection source, return max depth +1
            return max(leftDepth, rightDepth) + 1
        
        # If infection source is in one subtree, calculate max spread distance
        res = max(res, abs(leftDepth - rightDepth))
        
        # Return the negative depth to propagate infection path upwards
        return min(leftDepth, rightDepth) - 1
    
    DFS(root, start)  # Start DFS from the root node
    return res  # Return the maximum infection spread time
root = [1,5,3,None,4,10,6,9,2]
start = 3
amountOfTime(to_tree(root), start)

4

### Hard
#### [297. Serialize and Deserialize Binary Tree](https://leetcode.com/problems/serialize-and-deserialize-binary-tree/description/)

In [ ]:
#DFS preorder with 'n'
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Codec:
    def serialize(self, root):
        #add null markers # to uniquelt serialize a tree
        if not root:
            return '#'
        return str(root.val) + " " + self.serialize(root.left) + " " + self.serialize(root.right)
    def deserialize(self, s):
        array = s.split(" ")
        def dfs(array):
            val = next(array) #every time call next, consumed one
            if val == '#':
                return None
            left = dfs(array) #left will consume and the remaining is fed to right
            right = dfs(array) #right node keep consume array
            return TreeNode(val, left, right)
        return dfs(iter(array)) #call iter(array), to make pop first element instantly

# Your Codec object will be instantiated and called as such:
# ser = Codec()
# deser = Codec()
# ans = deser.deserialize(ser.serialize(root))


## Tries

### Med

#### [208. Implement Trie (Prefix Tree)](https://leetcode.com/problems/implement-trie-prefix-tree/submissions/)

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {} #children hashmap
        self.isWord = False
class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word: str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur = cur.children[c] #move to appropriate child node
        cur.isWord = True
    def search(self, word: str) -> bool:
        cur = self.root
        for c in word:
            if c not in cur.children:
                return False
            cur = cur.children[c]
        #after check if last TrieNode isWord after consuming word
        if not cur.isWord:
            return False
        return True
    def startsWith(self, prefix: str) -> bool:
        cur = self.root
        for c in prefix:
            if c not in cur.children:
                return False
            cur = cur.children[c]
        return True

#### [211. Design Add and Search Words Data Structure](https://leetcode.com/problems/design-add-and-search-words-data-structure/)

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {} #children hashmap
        self.isWord = False
    def __repr__(self) -> str:
        return "Children: " + str(self.children.keys()) + str(self.isWord)
       
class WordDictionary:
    def __init__(self):
        self.root = TrieNode()
    def addWord(self, word: str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur = cur.children[c]
        cur.isWord = True
    def search(self, word: str) -> bool:
        print(self.root.children)
        def dfs(word, root):
            cur = root
            for i in range(len(word)):
                if word[i] == '.':
                    for child in cur.children:
                        if dfs(word[i+1:], cur.children[child]): return True
                    return False       
                else:
                    if word[i] not in cur.children:
                        return False
                    cur = cur.children[word[i]]
            
            return cur.isWord == True
        
        return dfs(word, self.root)
    
wordDictionary = WordDictionary()

op = ["WordDictionary","addWord","addWord","addWord","addWord","search","search","addWord","search","search","search","search","search","search"]
param = [[],["babaa"],["and"],["an"],["add"],["a"],[".at"],["bat"],[".at"],["an."],["a.d."],["b."],["a.d"],["."]]
res = []
for i in range(1, len(op)):
    o, p = op[i], param[i]
    if o == "addWord":
        res.append(wordDictionary.addWord(param[i][0]))
    elif o == "search":
        res.append(wordDictionary.search(param[i][0]))
print(res)


### Hard

#### [212. Word Search II](https://leetcode.com/problems/word-search-ii/)

Approach: Search all words in the list at once by traveling each square in board, then dfs from the square outwards to see if the dfs path could check any word in the list.
1. Add all searching words to a Trie.
2. Use "refs" attribute for each nodes to represents # of chars in words shared a part of Trie path
<img src="assets/wordsearchii.png" width=500px></img>

3. For each square in board, DFS each square to find most suitable words at once.
    - Start with DFS (r,c, root, ""). String "" is word gained so far. 
    - On DFS: DFS up down left right square to find if each direction (on board) matches with any of current node's children (on Trie). Each time move deeper in DFS, current node = appropriate child node.
    - If word gain so far isWord, add to res, remove from Trie.
Complexity:
- If we brute force without using a trie

In [27]:
class TrieNode:
    def __init__(self) -> None: 
        self.children = {}
        self.ref = 0
        self.isWord = False
class Trie:
    def __init__(self) -> None:
        self.root = TrieNode()
    def addWord(self, word:str) -> None:
        cur = self.root
        for c in word:
            if c not in cur.children:
                cur.children[c] = TrieNode()
            cur = cur.children[c]
            cur.ref += 1 #incre the child's refs
        cur.isWord = True
        
    def removeWord(self, word:str)->None:
        cur = self.root
        for c in word:
            cur = cur.children[c]
            cur.ref -=1 #decre the child's refs
        cur.isWord = False 
    
class Solution:
    def findWords(self,board: list[list[str]], words: list[str]) -> list[str]:
        mtrie = Trie()
        for word in words:
            mtrie.addWord(word)
        res, visit = set(), set()
        ROW, COL = len(board), len(board[0])
        
        def dfssearch(r, c, node, formingWord):
            if (r not in range(ROW) or c not in range(COL) #out of bound 
            or (r, c) in visit #visited
            or board[r][c] not in node.children #not the right char square
            or node.children[board[r][c]].ref < 1): #if already found and removed
                return
             
            visit.add((r,c))
            formingWord += board[r][c]
            node = node.children[board[r][c]]
            
            if node.isWord:
                res.add(formingWord)
                mtrie.removeWord(formingWord)
            
            dfssearch(r+1, c, node, formingWord)
            dfssearch(r-1, c, node, formingWord)
            dfssearch(r, c+1, node, formingWord)
            dfssearch(r, c-1, node, formingWord)
            
            visit.remove((r,c))
        for r in range(ROW):
            for c in range(COL):
                dfssearch(r,c, mtrie.root, "")
        return list(res)

board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
words = ["oath","pea","eat","rain"]

# Example usage
board = [
    ['C', 'A', 'T'],
    ['I', 'D', 'O'],
    ['N', 'O', 'M']
]

words = ["CAT", "TOM", "ADO", "MOM"]
print(Solution.findWords(Solution, board=board, words=words))

['CAT', 'ADO', 'TOM']


## Backtracking
<img src="assets/backtrackrec.png" width="250ptx"></img>

### Type: all combinations

**Template 1**.  
 with `is_valid` (partition palindrome) and `additional states`
```python
ans = []
def dfs(start_index, path, [...additional states]):
    if is_leaf(start_index):
        ans.append(path[:]) # add a copy of the path to the result
        return
    for edge in get_edges(start_index, [...additional states]):
        # prune if needed
        if not is_valid(edge):
            continue
        path.add(edge)

        if additional states:
            update(...additional states)
        dfs(start_index + len(edge), path, [...additional states])
        # revert(...additional states) if necessary e.g. permutations

        path.pop()
```

#### [46. Permutations](https://leetcode.com/problems/permutations/)
<img src="assets/46. Permutations.png" width="300px"> </img>

Optimal time+space approach:
- Get perms from remain (nums.pop(0)), then append n=nums.pop(0) to each perm, extend perms to res
- End of the loop: nums.append(n) # because nums is passed as reference, we have to append n to the back. Or else nums would be pop all -> can't move to next of " for _ in nums:"



In [ ]:
class Solution:
    def permute(self, nums: list[int]) -> list[list[int]]:
        # print(nums)
        res = []
        if len(nums) == 1:
            return [nums[:]]

        for _ in nums:
            n = nums.pop(0)
            perms = self.permute(self,nums)
            for perm in perms:
                # print("append n", n)
                perm.append(n)
            # print("res ext", res, perms)

            res.extend(perms)

            nums.append(n) # because nums is passed as reference, we have to append n to the back
        return res
nums = [1,2,3,4,5]
print(Solution.permute(Solution, nums))

In [ ]:
class Solution:
    def permute(self, nums: list[int]) -> list[list[int]]:
        res = []
        def dfs(cur, nums):
            if not nums:
                res.append(cur[:])
                return
            # Explore all possibilities by adding each element in remaining to cur
            for num in nums:
                if num not in cur: #contraints shows we only have 6-10 nums, each num unique -> constant
                    cur.append(num)
                    dfs(cur, nums)
                    cur.pop()
        dfs([], nums)
        return res

#### [78. Subsets](https://leetcode.com/problems/subsets/)

<img src="assets/l78.jpg" width=400px></img>

In [ ]:
%%time
class Solution:
    def subsets(self, nums: list[int]) -> list[list[int]]:
        res = [[]]        
        def dfs(nums, subset):
            
            for i in range(len(nums)):
                subset.append(nums[i])
                
                res.append(subset[:])
                dfs(nums[i+1:], subset)

                subset.pop()
         
        dfs(nums, [])

        return res
nums = [n for n in range(1,12)]
Solution.subsets(Solution, nums)

Neet approach:
DFS decision tree to choose either append or not append nums[i]
<img src="assets/subset.png" width=400px></img>

In [ ]:
%%time
class Solution1:
    def subsets(self, nums: list[int]) -> list[list[int]]:
        res = []
        subset = []
        def dfs(i):
            #base case: reach the end of decision tree (completed set)
            if i >= len(nums):
                res.append(subset[:]) #add copy to keep modifying subset
                return 
            
            #decision to choose nums[i]
            subset.append(nums[i])
            dfs(i+1)
            #decision not to choose nums[i]
            subset.pop()
            dfs(i+1) #go to next level(i+1) without nums[i]
        dfs(0)
        return res
nums = [n for n in range(1,12)]
print(Solution1.subsets(Solution1, nums))

#### [39. Combination Sum](https://leetcode.com/problems/combination-sum/)
The backtracking decision tree to avoid duplicates candidates:
- Suppose decision paths through candidates = [2,3,6,7]
- Keep move from i, i or to i+1 in candidates, don't look back to avoid duplicates
    - The first paths that have first pivot elements, later path never contains previous pivot elements 
    - (Path of 3 never contains 2, path of 7 never contains 2,3,...)

<img src="assets/39. Combination Sum.png" width=500px></img>

**Note:  because we want to pass i+1 to be processed (the process of next index will be at line (I)), not process it now. "To-be-process" so that i+1 is treated like i, where it got processed, and its +1 (i+2 this case) got processed later. This recursively occur to i+2 to be processed so i+3 will be processsed after that.


In [ ]:
#we only move forward, never repeat previous index bc `index` variable is the left boundary block that
class Solution:
    def combinationSum(self, candidates: list[int], target: int) -> list[list[int]]:
        res = []
        n = len(candidates)
        def backtrack(index, cur, total):
            if total == target:
                res.append(cur[:]) 
                return
            if total > target:
                return
            #Loop from INDEX, to n => always move forward
            for i in range(index, n):
                cur.append(candidates[i])  # Include the candidate
                # Process updated total and the same index `i` (can reuse the same candidate)
                backtrack(i, cur, total + candidates[i])
                # Backtrack by removing the last added element
                cur.pop()
        
        backtrack(0, [], 0)
        return res
#Time: O(N^ (M/T) +1  ): Max nodes of depth of the tree:T/M 
#Space:(M/T) number of recursive calls

In [ ]:
class Solution:    
    def combinationSum(self, candidates: list[int], target: int) -> list[list[int]]:
        res = []
        cur = []
        #i: index of num in candidates; cur: current comb
        #(II): This function process i before new added cur, and added total
        #And process its next index i+1 that never consider i again
        def dfs(i, total):
            if total == target:
                res.append(cur[:])
                return
            if total > target or i >= len(candidates):
                return
            
            cur.append(candidates[i])
            #Process stage: pass i itself after the first pass at (II)
            #Process i with new added cur and new total

            dfs(i, total+candidates[i]) #(I)
            
            #Next index stage: pass i+1, would never consider i
            cur.pop() #pop what just added (candidates[i])
            
            #Note** Do not add candidates[i+1] to total
            dfs(i+1, total) 
            return
        dfs(0, 0)
        return res
candidates = [2,3,6,1]
target = 7
Solution.combinationSum(Solution, candidates, target)


#### [40. Combination Sum II](https://leetcode.com/problems/combination-sum-ii/description/s)

In [5]:
class Solution:
    def combinationSum2(self, candidates: list[int], target: int) -> list[list[int]]:
        res = []

        n = len(candidates)
        candidates.sort()
        def dfs(index, total, cur):
            if total == target:
                res.append(cur[:])
                return
            
            if index >= n or total > target:
                return
            
            # Explore each candidate starting at 'index'
            for i in range(index, n):
                # Skip duplicates: if the current candidate is the same as the previous one 
                # AND we're at the same recursive level (i.e., i - 1 >= index),
                # then skip this candidate to avoid duplicate combos. Already DONE building a comb start with that num.
                if i > index and candidates[i] == candidates[i - 1]:
                    continue
                
                # Choose candidates[i] and move forward
                cur.append(candidates[i])
                # Move 'i+1' because each number can be used only once
                dfs(i + 1, total + candidates[i], cur)
                # Backtrack: remove the last added number
                cur.pop()
        
                
        dfs(0,0, [])
        return res
#Time: 2^N: each number is included or excluded in a combination.
candidates = [2,3,2, 6,1]
target = 7
Solution.combinationSum2(Solution, candidates, target) 

[[2, 3, 2], [6, 1]]

#### [79. Word Search](https://leetcode.com/problems/word-search/)
O(n * m * 4^n)

- For time optimization: reverse the word if frequency of the first letter is higher than the last
- This reduces DFS search from multiple same chars in board in the beginning.

In [ ]:
from collections import Counter
class Solution:
    def exist(self, board: list[list[str]], word: str) -> bool:
        visit = set()
        ROW, COL = len(board), len(board[0])
        
        def dfs(r,c, word)->bool:
            #BASE CASE of TRUE/FALSE
            if len(word) ==0:
                return True
            
            if ((r,c) in visit
                or not (0 <= r and r <ROW)
                or not (0 <= c and c <COL) 
                or board[r][c] != word[0]):
                return False
            
            visit.add((r,c))
            print("found next at", board[r][c])

            if (dfs(r+1, c, word[1:])
            or dfs(r-1, c, word[1:])
            or dfs(r, c+1, word[1:])
            or dfs(r, c-1, word[1:])):
                return True

            visit.remove((r,c))
            return False
        #Reverse if count[word[0]] > count[word[-1]]
        c = Counter(word)
        if c[word[0]] > c[word[-1]]:
            word = word[::-1]
        
        for i in range(ROW):
            for j in range(COL):
                if dfs(i, j, word):
                    return True
        
        return False
board = [["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]]
word = "ABCCED"
Solution.exist(Solution, board, word)

found next at A
found next at B
found next at C
found next at C
found next at E
found next at D


True

#### [131. Palindrome Partitioning](https://leetcode.com/problems/palindrome-partitioning/)

OG approach:
- Pass in word to partition
- Loop i-> wordlen: Check if word[:i] is Pali, if yes: pass Parti(word[i+1:])

<img src="assets/131. Palindrome Partitioning.png" width=400px></img>
<img src="assets/131. PP.png" ></img>

In [130]:
def partition(s: str) -> list[list[str]]:
    result, path = [], []
    n = len(s)
    def partition(startIndex):
        if startIndex == len(s):
            result.append(path[:])
            return
        for end in range(startIndex, n):
            candidate = s[startIndex:end+1]
            if candidate == candidate[::-1]:
                path.append(candidate)
                # print(path)
                partition(end+1) #keep go deep from end pointer
                path.pop()
        # print(result)
        return
    partition(0)
    return result

partition("abcdcb") 

[['a', 'b', 'c', 'd', 'c', 'b'], ['a', 'b', 'cdc', 'b'], ['a', 'bcdcb']]

#### [17. Letter Combinations of a Phone Number](https://leetcode.com/problems/letter-combinations-of-a-phone-number/)

In [ ]:
def letterCombinations(digits: str) -> list[str]:
    #first thought: do for nested loop, but don't know how many times => backtracking
    #2 3 4 5
    mmap = {"2": "abc", "3": "def", "4":"ghi", "5":"jkl", "6":"mno", "7": "pqrs", "8":"tuv", "9":"wxyz"}

    def backtracking(digits):
        if len(digits) == 0:
            return []
        if len(digits) == 1:
            return list(mmap[digits[0]])
        cur = backtracking(digits[1:])
        temp = []
        for comb in cur:
            for letter in list(mmap[digits[0]]):
                temp.append(letter+ comb)

        return temp
    return backtracking(digits)
letterCombinations("567")
    

In [ ]:
def letter_combinations_of_phone_number(digits: str) -> list[str]:
    #mapping of number - letters
    phoneMap = {"2": ["a","b","c"], "3": ["d","e","f"], "4": ["g","h","i"], 
               "5": ["j","k","l"], "6": ["m","n","o"], "7": ["p","q","r","s"], 
               "8": ["t","u","v"], "9": ["w","x","y", "z"] }
    # apply algomonster backtracking template
    arr = []
    digits = list(digits)
    def dfs(comb, n, arr, remainDigits):
        if len(comb) == n:
            arr.append("".join(comb))
            return
        for letter in phoneMap[remainDigits[0]]:
            comb.append(letter)
            dfs(comb, n, arr, remainDigits[1:])
            comb.pop()
            
    dfs([], len(digits), arr, digits)
    return arr
letter_combinations_of_phone_number("567")

#### [90. Subsets II](https://leetcode.com/problems/subsets-ii/)

In [ ]:
class Solution:
    def subsetsWithDup(self, nums: list[int]) -> list[list[int]]:
        res = []
        cur = []
        nums.sort()
        def dfs(i):
            if i >= len(nums):
                
                res.append(cur[:])
                return
            
            #if choose nums[i], go dfs
            cur.append(nums[i])
            dfs(i+1)
            #if not choose nums[i]
            cur.pop()
            count = 0
            #after pop(), don't dfs nums[i+1] if , skip i to end of same element sequence
            while i+1 < len(nums) and nums[i] == nums[i+1]:
                count+=1
                i+=1
            
            #dfs without num[i]
            dfs(i+1)
            
        dfs(0)
        
        return res
nums = [1,2,3,2,2,2,3,4,4]
# print(Solution.subsetsWithDup(Solution, nums))    

### Aggregation and memoization
The main differences w/ previous template :
- no path and push/pop, no generate the solutions, just need the aggregated value.
- use return value to aggregate results from child dfs calls.

```python
def dfs(start_index, [...additional states]):
    if is_leaf(start_index):
        return 1
    ans = initial_value
    for edge in get_edges(start_index, [...additional states]):
        if additional states: 
            update([...additional states])
        ans = aggregate(ans, dfs(start_index + len(edge), [...additional states]))
        if additional states: 
            revert([...additional states])
    return ans
```

1. Memoization:
```python
def fib(n, memo):
    if n in memo: # check in memo, if found, retrieve and return right away
        return memo[n]

    if n == 0 or n == 1:
        return n

    res = fib(n - 1, memo) + fib(n - 2, memo)

    memo[n] = res # save in memo before returning
    return res
```

#### [139. Word Break](https://leetcode.com/problems/word-break/description/)

In [ ]:
#Explain on how memo helps - multiple way to reach dfs(x), don't recompute
'''
s = "pineapplepenapple"
wordDict = ["pine", "pineapple", "apple", "pen", "applepen"]

To reach startIndex = 12 (suffix "apple"):
0: "pineapple" → 9, then "pen" → 12
0: "pine" → 4, "apple" → 9, "pen" → 12
0: "pine" → 4, "applepen" → 12
0: "pineapple" → 9, "pen" → 12
Once you compute dfs(12), never want to recompute it.
--
Memo matters most when:
the string is not breakable, or
the breakable path is found late, after trying many failing branches
--
That's when DFS explores lots of branches and revisits the same dfs(i) states.
s = "pineapplepenapplx"     # ends with x, impossible
wordDict = ["pine", "pineapple", "apple", "pen", "applepen"]

At index 0 it tries words in some order. Suppose it tries:
now you call dfs(12) on suffix "applx"
it explores possibilities, eventually returns False
backtrack to 9: try other options (none), return False
backtrack to 4: try "applepen" → index 12
now you call dfs(12) again on the same suffix "applx"
without memo you re-explore the same failing work again
'''

In [ ]:
def word_break(s: str, wordDict: list[str]) -> bool: #with memo
    n = len(s)
    memo = {}
    def dfs(startIndex):
        if startIndex in memo:
            return memo[startIndex]
        if startIndex == n:
            memo[startIndex] = True
            return True
        for word in wordDict:
            if s[startIndex:].startswith(word): 
                if dfs(startIndex+len(word)): 
                    #once we compute dfs(x), don't compute again, save to memo
                    memo[startIndex+len(word)] = True
                    return True
        memo[startIndex] = False
        return False
    return dfs(0)

s = "catsandog"
wordDict = ["cats","dog","sand","and","cat"]
s = "algomonsterplplalmonsterplpl"
wordDict = ["algo", "monster", "plpl"]
word_break(s, wordDict)


False

Iterative explain 
**What `prev1` and `prev2` mean**

* `prev1` = how many full decoding ways exist **1 step before i**
* `prev2` = how many full decoding ways existed **2 step before i**

They are **counts**, not steps.

`curr += prev1`: If the current digit can stand alone, then **every existing decoding** (`prev1` of them) can extend.

`curr += prev2`: If the last two digits can be decoded together, then **every decoding from two steps ago** (`prev2` of them) can extend.


In [ ]:
#solve regular first, memo later
'''
memo, what to save, what can be repeated and reuse 
"1|8|2|4|120109"
"1|8|2|4|120109"
repeated like this: when we check if digits.startwith(letter), like 1824120109 will go 2 around in for loop:
round 1: 1 + dfs(824120109), 8+dfs(24120109); round2: 18+dfs(24120109)-> dfs(24120109) can computed only once.

count
"1|2|312","1|2312", "12|312"
1 2 3 1 2 -> one way
1 2 3 1 (2) count + 1 -> return count = 1
1 2 3 dfs(1 2) count + 1 with (1 - 2), count +=1 with (12) -> count = 2; return 2

1 2 dfs(3 1 2) return 2 

123324-> 1 2|12, 2 2|22
1 2 22, 12 22, 1 2 2
'''
def decode_ways_dfs(digits: str) -> int:
    memo = {}
    n = len(digits)
    def dfs(startIndex):
        count = 0
        if startIndex in memo:
            return memo[startIndex]
        if startIndex == n:
            return 1
        
        #for each dfs round, we can only do 3 digit analyze: 
        # path with leading 0 is not possible, immediately return
        if digits[startIndex] == '0':
            return 0
        #process 1 digit
        count+=dfs(startIndex+1)
        if 10 <= int(digits[startIndex:startIndex+2]) < 27: 
            count+=dfs(startIndex+2)
        memo[startIndex] = count
        return count
    return dfs(0)

'''
prev1 = ways to decode s[:i] (prefix ending at i-1)
prev2 = ways to decode s[:i-1] (prefix ending at i-2)
curr = ways to decode s[:i+1] (prefix ending at i)
like: 
given 1232425, 
pathto4= path(123)(prev2)+24, + path(1232)(prev1)+4 (2 totally different path so far)
pathto2 = |pathto4(prev1) + 2| + x|(pathto4-1stepback)(prev2) + 42(but no valid tho)|
path to 5 = pathto2 +...
'''
def decode_ways(digits: str) -> int:
    if digits[0] == '0':
        return 0
    n = len(digits)
    prev1 = 1 # one way available for 1 step back
    prev2 = 1 #one way available for 2 stepback
    #start i = 1, because we want to examine 2digit inlcuding i=1, and i=0
    for i in range(1,len(digits)):
        cur = 0 #path-to-i
        if digits[i] != '0': #1020312
            #then we can add i to prev1
            cur+=prev1
        if 9<int(digits[i-1:i+1])<27:
            cur+=prev2
        prev2 = prev1 #in the next i, step oneback become step two back
        prev1 = cur #and current is step oneback.
    return prev1
#todo dp back here

In [ ]:
# 322. Coin Change
def coin_change(coins: list[int], amount: int) -> int:
    #start with largest coin to smallest coin
    memo = [-2]*(amount+1)
    # print(memo)
    #dfs to find minCount for amount x
    def dfs(amount, memo, coins): #memo, coins passed just for quicker memory access
        if memo[amount] != -2:
            return memo[amount]
        if amount == 0:
            return 0
        minCoin = float('inf')
        for coin in coins:
            if coin <= amount: 
                temp = dfs(amount-coin, memo, coins)
                if temp != -1: #can choose coin here and go down path(dfs(am-coin)) 
                    minCoin = min(1+temp, minCoin) #picked "coin"+ dfs(am-coin)
        memo[amount] = minCoin if minCoin!=float('inf') else -1
        return memo[amount] 
    return dfs(amount, memo, coins)
#todo dp back here (dp and variables only)
coins = [186, 419, 83, 408]
amount = 6249
coin_change(coins,amount)

20

### Deduplication

In [ ]:
# 3sum
def three_sum_unique_triplets(nums: list[int], target: int) -> list[list[int]]:
    nums.sort()
    res = []

    # use two pointers, but be aware of duplicates while moving pointers
    def twoSum(nums, target):
        res = []
        l, r = 0, len(nums)-1
        while l < r:
            cursum = nums[l] + nums[r] 
            # be aware of duplicates while moving pointers
            if cursum > target:
                while nums[r] == nums[r-1]:
                    r-=1
                r-=1
            elif cursum < target:
                while nums[l] == nums[l+1]:
                    l+=1
                l+=1
            else:
                res.append([nums[l], nums[r]])
                l+=1
                r-=1
        return res
    
    #pin i, then find twoSum([remain], target-nums[i])
    for i in range(len(nums)):
        if i > 0 and nums[i] == nums[i-1]: #pass duplicated
            continue
        twoSumList = twoSum(nums[i+1:], target-nums[i]) # dfs go one step ahead and never look back
        if len(twoSumList)>0:
            for list in twoSumList: list.append(nums[i])
            res.extend([twoSumList])
        
    return res
nums = [0,1, 2, 3, 4, 5] 
target = 8
nums = [1, 1, 2, 3, 4, 5] 
target = 6
three_sum_unique_triplets(nums, target)

[[1, 4], [2, 3]]


[[[1, 4, 1], [2, 3, 1]]]

2056. Number of Valid Move Combinations On Chessboard

In [ ]:
from typing import List

class Solution:
    def countCombinations(self, pieces: List[str], positions: List[List[int]]) -> int:
        # Define movement directions for each piece type
        d_rook = [(1, 0), (-1, 0), (0, 1), (0, -1)]  # Vertical & horizontal moves
        d_bishop = [(1, 1), (1, -1), (-1, 1), (-1, -1)]  # Diagonal moves
        d_queen = d_rook + d_bishop  # Queen can move like both rook and bishop
        
        # Assign movement directions based on piece type
        directions = [
            d_rook if piece == 'rook' else 
            d_bishop if piece == 'bishop' else 
            d_queen for piece in pieces
        ]
        
        # Define board size and max steps allowed for movement
        m = n = max_step = 8  # Standard chessboard size
        
        # Track occupied steps for each position on the board
        occupied_by_step = [[[] for _ in range(n)] for _ in range(m)]
        end_step = [[max_step] * n for _ in range(m)]
        
        def backTrack(i):
            """Recursive backtracking function to explore all move combinations."""
            if i == len(pieces):  # Base case: all pieces have moved
                return 1
            
            res = 0  # Count of valid move combinations
            x, y = positions[i][0] - 1, positions[i][1] - 1  # Convert position to 0-based index
            
            # Case 1: The piece does not move
            if end_step[x][y] == max_step and not occupied_by_step[x][y]:
                end_step[x][y] = 0  # Mark as occupied at step 0
                res += backTrack(i + 1)
                end_step[x][y] = max_step  # Reset after backtracking
            
            # Case 2: The piece moves in one of its allowed directions
            for r, c in directions[i]:
                x, y = positions[i][0] - 1, positions[i][1] - 1  # Reset position
                visit = []  # Track visited positions
                
                for step in range(1, max_step):  # Move up to max_step
                    x, y = x + r, y + c  # Move in the given direction
                    
                    # Check if the move is within bounds and does not conflict with another piece
                    if 0 <= x < m and 0 <= y < n and step not in occupied_by_step[x][y] and step < end_step[x][y]:
                        last_occupied_step = max(occupied_by_step[x][y] or [0])  # Track last occupied step
                        occupied_by_step[x][y].append(step)  # Mark step as occupied
                        visit.append((x, y))
                        
                        # Stop at this position and proceed with the next piece
                        if end_step[x][y] == max_step and step > last_occupied_step:
                            end_step[x][y] = step
                            res += backTrack(i + 1)
                            end_step[x][y] = max_step  # Reset after backtracking
                    else:
                        break  # Stop moving in this direction
                
                # Backtrack: Remove occupied steps from visited positions
                for x, y in visit:
                    occupied_by_step[x][y].pop()
            
            return res
        
        return backTrack(0)  # Start exploring moves from the first piece


## [Heap / Priority Queue](https://leetcode.com/problems/kth-largest-element-in-a-stream/)

- **Heap**: A tree-based, implement PQ (often stored in an array) where each parent satisfies (parent ≤ children min-heap).  
- **Left, right, parent child**: \(2i + 1\), \(2i + 2\), (i - 1)//2 
- **Push** and **pop** in **O(\(\log n\))**
- **Common Patterns**:  
  - **Top K** problems (kth largest or smallest)  
      - Lots of Kth problems can be solved by 
         - **Quick Select**, or modified **binary search on selected threshold for unsorted array** 
         - avg O(n) by roughly halving the total number each iteration, worst O(n^2)

  - **Merging** multiple sorted lists or data streams  
  - **Scheduling** or event simulation (process highest or lowest priority first)  
  - **Shortest Path** algorithms (Dijkstra’s) where you repeatedly extract the smallest distance  
- Note: for lots of heap problems of kth, we want to init heap to have k size that fills -inf elements if heap.size < k to avoid later edge cases.

In [ ]:
import heapq

heap = []            # creates an empty heap
heappush(heap, item) # pushes a new item on the heap
item = heappop(heap) # pops the smallest item from the heap
item = heap[0]       # smallest item on the heap without popping it
heapify(x)           # transforms list into a heap, in-place, in linear time
item = heappushpop(heap, item) # pushes a new item and then returns
                               # the smallest item; the heap size is unchanged
item = heapreplace(heap, item) # pops and returns smallest item, and adds
                               # new item; the heap size is unchanged

In [ ]:
class MinHeap:
    """
    A min-heap ensures the smallest element is always at the root (index 0).
    """

    def __init__(self):
        self.heap = []  # The heap is stored in a list.

    # Helper methods to get parent and children indices:
    def parent(self, i):
        return (i - 1) // 2

    def left_child(self, i):
        return 2 * i + 1

    def right_child(self, i):
        return 2 * i + 2

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def heapify_up(self, i):
        """
        'heapify_up' moves a newly added element upwards if it's smaller
        than its parent—fixing violation from bottom to top.
        """
        # While we haven't reached the root AND the current item is smaller than its parent:
        while i > 0 and self.heap[i] < self.heap[self.parent(i)]:
            # Swap with its parent to move the smaller item up.
            self.swap(i, self.parent(i))
            # Update 'i' to continue checking higher up the heap.
            i = self.parent(i)

    def heapify_down(self, i):
        """
        'heapify_down' moves an element downwards if it's larger than one
        of its children—fixing violations from top to bottom.
        """
        n = len(self.heap)
        while True:
            smallest = i  # Assume the parent is smallest initially.
            left = self.left_child(i)
            right = self.right_child(i)

            # Check if left child is smaller.
            if left < n and self.heap[left] < self.heap[smallest]:
                smallest = left

            # Check if right child is even smaller.
            if right < n and self.heap[right] < self.heap[smallest]:
                smallest = right

            # If the smallest is not the current node, swap and continue down.
            if smallest != i:
                self.swap(i, smallest)
                i = smallest
            else:
                # If the current node is already smaller than both children, stop.
                break

    def heapify(self, array):
        """
        'heapify' builds a heap out of an entire array at once by
        calling 'heapify_down' on non-leaf nodes, from right to left.
        This is more efficient (O(n)) than inserting items one by one.
        """
        self.heap = array  # Initialize the heap with the given array.

        # Start from the last non-leaf node down to the root.
        # For each, push it down if necessary to satisfy min-heap rules.
        for i in range(len(self.heap) // 2 - 1, -1, -1):
            self.heapify_down(i)

    def push(self, item):
        """
        Insert 'item' into the heap:
          1) Append to the end.
          2) 'heapify_up' to fix any violations from bottom to top.
        """
        self.heap.append(item)
        self.heapify_up(len(self.heap) - 1)

    def pop(self):
        """
        Remove and return the smallest element (root).
          1) Swap root with the last element, pop it.
          2) 'heapify_down' from the root to re-heapify.
        """
        if len(self.heap) == 0:
            return None

        root = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        self.heapify_down(0)
        return root


Note: 
- Set of k largest: Use min heap size k of the largest set, heap[0] is the smallest, used as boundary.
- Set of k smallest: k largest but negative val.

### Easy

#### [703. Kth Largest Element in a Stream](https://leetcode.com/problems/kth-largest-element-in-a-stream/)

Approach:
- Use min-heap of size k. 
    -  CREATE K-SIZE HEAP by appending from the largest in nums

nums = [2,3,4,5,6], k =3 -> minheap = [4,5,6], heap[0] = 4

- If (adding val to nums) val > heap[0], minheap.push(val), minheap.pop(). Return heap[0]

In [80]:
import heapq
class KthLargest:
    def __init__(self, k: int, nums: list[int]):
        self.heap = nums
        heapq.heapify(self.heap)
        while len(self.heap) > k:
            heapq.heappop(self.heap)
        
        #add -inf to heap if size heap < k, so that we'll don't have to deal with edge case later in add()
        if len(self.heap) < k: 
            heapq.heappush(self.heap, float('-inf')) #-> any next val can be pushed to heap
            

    def add(self, val: int) -> int:
        #must be larger than the smallest in the upperlarge-heap to join heap
        if val > self.heap[0]:
            heapq.heappop(self.heap)
            heapq.heappush(self.heap, val)
        return self.heap[0]
        
            
kthLargest = KthLargest(3, [4, 5, 8, 2])
print(kthLargest.add(3))
print(kthLargest.add(5))
print(kthLargest.add(10))
print(kthLargest.add(9))
print(kthLargest.add(4))

4
5
5
8
8


#### [1046. Last Stone Weight](https://leetcode.com/problems/last-stone-weight/)

- Heapify -stones for maxheap
- Keep heappop the largest two elements in heap -> compare -> decide to heappush different largest - second

In [ ]:
import heapq
#work on negative sign heap -> change sign all operations
def lastStoneWeight(stones: list[int]) -> int:
    heap = [-x for x in stones]
    
    heapq.heapify(heap)
    
    while len(heap) >1:
        largest = heapq.heappop(heap)
        #largest and second large comparison
        second = heapq.heappop(heap)
        if largest != second:
            heapq.heappush(heap, largest - second)
    return 0 if not heap else -heap[0]
    
stones = [2,7,4,1,8,1]
lastStoneWeight(stones)

1

#### [973. K Closest Points to Origin](https://leetcode.com/problems/k-closest-points-to-origin/)
- Create min heap size k, only contains k largest -distance (smallest real distance)
- Heap form = [(val, [key])]

In [94]:
import heapq
def kClosest( points: list[list[int]], k: int) -> list[list[int]]:
    heap = [[float('-inf'), 0, 0]]*k
    #maxheap to store min distance, root is max of kth min distances
    for p in points:
        dis = p[0]**2 + p[1]**2

        if -dis > heap[0][0]:
            heapq.heappush(heap, [-dis, p[0], p[1]])
            heapq.heappop(heap)
    
    return [[x,y] for [z, x, y] in heap]

points = [[3,3],[5,-1],[-2,4]]
k = 2
kClosest(points, k)

[[-2, 4], [3, 3]]

**Modifed binary search approach** (similar to Quick select).
- **Algorithm**: On each iteration, pick a distance `mid` and split points into two groups: those within `mid` (closer) and those beyond `mid` (farther).  
- **Discard Step**: Depending on the size of the “closer” group relative to \(k\), we discard one group entirely (either the closer group if it’s too big, or the farther group otherwise).  
- **Halving Effect**: Because we throw away about half the points on each partition in the average case, the total work sums up to about \(O(n)\) overall 

In [134]:
#modify binary search
import math
def kClosest(points, k):
    #get the mid threshold from 0 and max(distances)
    #divide points to closer and farther halves, work until len(closer)==k
    
    dist = [point[0]**2+point[1]**2 for point in points ]
    lo, hi = 0, max(dist)
    #remain points to work with, after each iteration of halving distances
    remainPoints = [i for i in range(len(points))]
    totalClose = []
    while True:
        mid = (hi + lo)/2
        close, far = splitPoints(remainPoints, dist, mid)
        
        if len(close) == k:
            totalClose.extend(close)
            return [points[i] for i in totalClose]
        elif len(close) < k:
            #move mid farther
            lo = mid
            k -= len(close) #remain k to be added after count all close points within k
            totalClose.extend(close) # add to total
            remainPoints = far

        else:
            #ignore farther
            hi = mid
            remainPoints = close
            
def splitPoints(remain, distance, mid):
    close, far = [], []
    for idx in remain:
        if distance[idx]> mid:
            far.append(idx)
        else:
            close.append(idx)
    return [close, far]

    
kClosest([[0,1],[1,0]],2)

[[0, 1], [1, 0]]

#### 215. Kth Largest Element in an Array

Heap aproach: Avg + Worst: Nlogn

- Heapify nums -> pop k times

In [ ]:
def findKthLargest(nums: list[int], k: int) -> int:
    nums = [-i for i in nums]
    heapq.heapify(nums)
    for i in range(k-1):
        heapq.heappop(nums)
    return -nums[0]
nums = [3,2,1,5,6,4]
k = 2
findKthLargest(nums, k)

5

Quicksort descending new: most efficient - 
- Random pivot
- Create 3 arrays: left(all nums > pivot), mid(nums==pivot), right(nums< pivot)
- if k <= len(left) => recursively go to left
- if k > len(left)+len(mid) => recursively go to right
- else return mid[0], k is the pivot(s)

In [135]:
import random
def findKthLargest(nums: list[int], k: int) -> int:
    pivot = random.choice(nums)
    left = [num for num in nums if num > pivot]
    right = [num for num in nums if num < pivot]
    mid = [num for num in nums if num == pivot]
    
    if k <= len(left): return findKthLargest(left, k)
    if k > len(left)+len(mid): return findKthLargest(right, k-len(left)-len(mid))
    return mid[0]

Approach: Old QuickSelect Pivot - RTL error
- Avg: O(n), worst O(n^2)
- Quick sort until kth pivot, quicksort is recursive
- Quicksort(l,r) (start with 0, len(nums))
    - Init: **pivotNumber, pivotPointer** = nums[r], l
    - for i in range(l, r):
        - If nums[i] >= nums[pivot], swap(num[i], num[p]); p+=1 
            - i+=1 (**-> p reserve the next spot for the next larger**))
    - swap nums[p], nums[pivot](**After the loop, p always reserve the spot to where pivot should be**)
    

In [137]:
def findKthLargest(nums: list[int], k: int) -> int:
    def quickShort(l,r):
        p, pivot = l, nums[r]
        for i in range(l,r):
            #even don't have to swap for "=" case, still use
            #"=" to p++, reserve spot for next ">=" nums[i]       
            if nums[i] >= pivot:
                nums[i], nums[p] = nums[p], nums[i]
                p+=1
        nums[p], nums[r] = pivot, nums[p]
        #if p < k, go to larger partition: the right
        if p < k: return quickShort(p+1, r)
        if p > k: return quickShort(l, p-1)
        return nums[p]
    return quickShort(0, len(nums)-1)

#### [621. Task Scheduler](https://leetcode.com/problems/task-scheduler/)
Intuition: Most frequent task processed first to minimize idle time ->
1. use **heap** to track most frequent **IN USE**task
2. use **queue** to track FIFO **WAITING TASK**
Think of tasks as their frequents: AAABBCC -> 3,2,2

Approach: Implement maxheap of counts of most frequent letters, use **queue** to FIFO waiting time - Form:[[count1:time1], [count2:time2],....]
- While **heap or q** #A letter on **ALWAYS- EITHER- wait** ||  **heap**
    - Each time used a letter => Heap[0]+=1, add to q, pop heap
    - If q[0].time == currentTime: add back to **IN USE** heap

In [138]:
import heapq
from collections import Counter, deque
def leastInterval(tasks: list[str], n: int) -> int:
    c = Counter(tasks)
    heap = [-x for x in c.values()]
    heapq.heapify(heap)
    q = deque()
    time = 0
    while heap or q: #A letter ALWAYS EITHER on wait or on heap
        time +=1

        #if not heap -> all letters IDLE in q
        #if heap, consume root(most freq letter)
        if heap:
            if heap[0]+1 != 0: #if there's still letters after consuming 1
                q.append([heap[0]+1, time+n]) #add to q end
            heapq.heappop(heap) #Used top letter
        
        #push back to heap the first letter ends to wait in IDLE
        if q and q[0][1] == time:
                heapq.heappush(heap, q.popleft()[0]) #pop IDLE, push count to heap    
    return time
    
tasks = ["B","F","J","J","H","A","D","C","C","D","J","E","B","E","C","H","E","E","G","E","H","I","I","E","H","F","C","G","H","F","E","E","I","D","D","A","E","A","C","E","F","J","E","F","G","J","A","A","A","B","E","J","H","C","A","E","D","A","E","I","H","B","A","C","E","F","J","D","F","I","I","H","J","E","B","H","H","A","J","C","C","D","C","B","C","H","H","I","H","D","C","B","D","C","H","A","F","A","J","D","F","E","H","I","D","A","E","B","H","J","F","D","C","J","J","I","A","I","J","H","A","I","I","G","C","C","H","D","B","B","B","H","I","D","B","C","H","I","I","G","E","D","D","A","E","D","H","C","J","H","C","E","I","F","A","I","E","G","A","E","F","I","B","J","B","J","B","G","A","D","C","B","I","A","C","J","J","J","B","E","E","E","B","E","B","H","B","J","H","F","G","B","B","I","C","D","I","D","D","A","I","D","H","A","J","D","J","D","I","G","F","F","B","G","A","F","I","I","H","C","B","H","H","E","F","A","H","G","F","D","G","F","A","C","F","D","F","C","J","A","D","H","G","D","D","G","C","G","C","I","B","G","E","J","C","G","J","G","E","F","F","I","D","D","E","G","F","A","F","H","D","C","B","E","H","G","H","B","A","D","A","J","A","C","B","G","C","A","J","E","I","G","H","J","A","J","I","D","I","G","B","G","I","A","B","J","B","H","E","G","G","I","G","G","C","J","J","J","H","G","A","G","B","G","I","G","A","D","B","A","E","I","H","E","G","H","J","F","H","I","A","E","A","I","A","D","D","B","D","G","E","C","J","C","C","B","F","F","B","E","A","G","D","I","G","D","J","D","G","I","B","J","J","B","F","D","I","C","I","B","J","B","D","D","A","D","C","G","A","G","C","G","C","E","C","G","A","J","C","E","F","C","C","E","J","D","H","H","F","E","D","G","E","E","I","B","G","A","C","C","E","J","F","H","C","A","G","H","G","A","G","A","D","B","E","F","H","C","C","B","I","F","H","I","C","H","G","G","C","E","A","J","F","F","F","B","D","I","E","I","F","E","F","G","G","A","D","I","A","J","A","H","J","I","H","H","A","B","B","B","I","C","J","C","G","J","G","E","I","H","D","E","H","J","A","G","A","G","A","C","F","C","G","E","G","J","D","H","H","B","J","F","J","J","B","J","D","D","G","B","I","I","H","G","B","I","E","D","H","H","I","B","A","A","A","D","H","J","H","D","J","E","D","G","J","B","B","F","G","J","G","E","G","E","E","F","H","F","J","A","G","I","D","C","H","A","C","F","B","J","E","I","A","G","J","D","I","F","I","J","H","E","D","J","A","E","G","B","B","J","I","J","H","F","D","F","F","A","G","F","F","I","C","H","E","E","G","A","D","F","I","D","D","C","B","G","C","D","G","H","F","D","J","D","B","A","J","J","I","H","F","G","D","J","H","I","H","I","A","A","I","C","B","H","G","I","C","F","E","J","J","F","F","D","F","A","J","H","B","G","A","F","D","G","D","C","F","J","F","G","D","H","J","C","A","E","C","G","J","G","I","C","G","H","G","J","D","D","G","D","F","F","J","B","D","C","E","F","G","D","A","J","H","D","F","C","B","H","C","I","D","C","F","E","C","D","J","D","E","G","C","D","H","J","E","H","I","I","A","C","E","C","I","B","A","B","E","E","H","E","B","H","C","G","B","C","C","D","G","G","A","F","A","B","D","G","F","A","H","G","C","E","D","B","H","D","F","F","G","A","J","H","E","B","C","B","B","B","H","D","F","B","B","C","G","A","C","E","J","H","F","F","D","G","J","D","F","J","G","H","B","D","B","D","D","G","J","H","B","D","F","E","E","G","D","H","B","A","I","E","B","E","B","D","I","C","A","A","E","J","A","B","A","F","C","J","F","F","F","A","I","J","F","H","G","C","F","E","D","C","B","C","G","G","G","A","B","J","J","F","J","G","C","D","B","C","F","H","I","F","D","C","I","J","D","I","G","B","G","I","J","E","H","G","G","J","J","A","E","E","I","G","E","H","F","F","C","A","J","I","I","D","C","E","G","A","A","H","F","A","I","B","H","J","H","B","H","A","A","A","G","I","I","D","F","F","C","H","D","B","J","F","G","E","F","J","A","I","G","J","J","F","F","G","B","B","D","F","E","G","D","D","A","D","D","G","C","C","I","C","H","I","C","E","G","C","E","J","F","G","J","B","I","B","B","C","B","G","A","J","H","C","G","D","E","H","E","H","A","H","I","A","J","C","G","B","G","H","G","H"]
n = 36
leastInterval(tasks, n)

4256

#### [355. Design Twitter](https://leetcode.com/problems/design-twitter/)

Notes:
- getNewsFeed(userId): O(n log k) - Where 'n' is the total number of tweets from the user and their followees, and 'k' is the number of tweets to return (10 in this case). This is due to the heapq.nlargest operation.

Attributes:  
*   **`follower`:** defaultdict(set)  
*   **`tweet`:** Dict map tweets for each user with timestamps, defaultdict(list)
*   **`time`:** Global counter to order tweets.  

Methods: 
*   **`postTweet`:** Adds a tweet with a timestamp.
*   **`getNewsFeed`:**
    *   Gathers tweets from the user and those they follow.
    *   Uses `heapq.nlargest` to get the 10 most recent.
*   **`follow` / `unfollow`:** Updates the follower relationships.

This structure allows for efficient retrieval of recent tweets by combining followed users' tweets and using a heap for the top 10.


In [140]:
from collections import defaultdict
tweets = defaultdict(list)
tweets["a"].extend([[3,2],[1,2],[4,5]])
v=tweets["a"][:]
v.append([99])
print(v, tweets["a"])

[[3, 2], [1, 2], [4, 5], [99]] [[3, 2], [1, 2], [4, 5]]


In [162]:
from collections import defaultdict
import heapq
class Twitter:

    def __init__(self):
        self.tweets = defaultdict(list)
        self.followers = defaultdict(set)
        self.time = 0

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.time+=1
        self.tweets[userId].append((tweetId,self.time)) #userID:[(tweetID, time)]
    def getNewsFeed(self, userId: int) -> list[int]:
        #Get all tweets from followers and user themselves
        validTweets = self.tweets[userId][:] #copy list of tweets to validtweets
        for followeeID in self.followers[userId]:
            validTweets.extend(self.tweets[followeeID])
            
        return [tweetID for tweetID,_ in heapq.nlargest(10, validTweets)]

    def follow(self, followerId: int, followeeId: int) -> None:
        self.followers[followerId].add(followeeId)

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followeeId in self.followers[followerId]:
            self.followers[followerId].remove(followeeId)

    def do(self, ops, params):
        self.tw = Twitter()
        res = []
        for i in range(len(ops)):
            if ops[i] == "Twitter":
                res.append(None)  # Initialization returns None
            elif ops[i] == "postTweet":
                res.append(self.tw.postTweet(*params[i]))
            elif ops[i] == "getNewsFeed":
                res.append(self.tw.getNewsFeed(*params[i]))
            elif ops[i] == "follow":
                res.append(self.tw.follow(*params[i]))
            elif ops[i] == "unfollow":
                res.append(self.tw.unfollow(*params[i]))
        return res
    def do(self,op, param):
        res = []
        for i in range(len(op)):
            if op[i] == "Twitter":
                continue
            if op[i] == "postTweet":
                res.append(self.postTweet(*param[i]))
            elif op[i] == "getNewsFeed":
                res.append(self.getNewsFeed(*param[i]))
            elif op[i] == "follow":
                res.append(self.follow(*param[i]))
            elif op[i] == "unfollow":
                res.append(self.unfollow(*param[i]))
        return res 

In [163]:
# TEST, put do inside twitter class
           
# op = ["Twitter", "follow","follow","follow","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","postTweet","getNewsFeed"]
# pa = [[],[1,5],[1,6],[1,7],[7,77],[1,505],[1,111],[5,555],[6,66],[1,5],[2,3],[3,101],[4,13],[1,22],[1,11],[1]]

op = ["Twitter","postTweet","getNewsFeed","follow","getNewsFeed","unfollow","getNewsFeed"]
pa = [[],[1,1],[1],[2,1],[2],[2,1],[2]]

t = Twitter()
t.do(op,pa)
#[None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,[222,204,200,201,205,11,333,94,2,101],None,[211,222,233,208,204,202,200,213,201,203],None,[222,204,200,201,205,11,333,94,2,101]]

[None, [1], None, [1], None, []]

#### [Meeting Rooms II](https://leetcode.com/problems/meeting-rooms-ii/editorial/)


In [235]:
#sort start time, then minheap for endtime
def minMeetingRooms( intervals: list[list[int]]) -> int:
        intervals.sort(key= lambda x: x[0]) # Sort meetings by start time.
        rooms = [] # heap contains ending times, imply rooms used

        # For all the remaining meeting rooms
        for start,end in intervals:
            # if earliest ending <= current start, free earliest ending from heap
            if rooms and rooms[0] <= start:
                heapq.heappop(rooms)

            # add new ending time
            heapq.heappush(rooms, end)
        
        #  size of the ending time heap implied minimum rooms required used, 
        #size of the heap (which stores end times) ultimately reflects the maximum number of rooms needed at any point in time
        # Because we don't pop when all meeting finish, only pop when one finish before another start (use the same room)
        #the code says: heap gets increment every time a new room allocated, and stays the same if previous meeting popped. heap reflects all room has been used all the time.
        return len(rooms)
minMeetingRooms([(0,5 ), (0,1), (0,2), (0,3), (5,10)])

MY HEAP,  [1, 5]
MY HEAP,  [1, 5, 2]
MY HEAP,  [1, 3, 2, 5]
MY HEAP,  [2, 3, 5, 10]


4

In [ ]:
#Approach 2: sort start and end time, use 2 pointers start and end
#keep track of roomusedCount, if start < end: new room allocated,else, use old room.
'''
When encounter an ending event => some meeting that started earlier has ended now. we don't care which meeting has ended. we know meeting ended thus making a room available.

'''
def minMeetingRooms(intervals: list[list[int]]) -> int:
        
        # If there are no meetings, we don't need any rooms.
        if not intervals:
            return 0
# [1,6] [2,5] [3,4] [ 2,3] => 1 2 2 3 , end 3 4 5 6 => 
        used_rooms = 0

        startTime = sorted([i[0] for i in intervals])
        endTime = sorted(i[1] for i in intervals)
        
        end_pointer = 0
        start_pointer = 0

        # Until all the meetings have been processed
        while start_pointer < len(intervals):
            # If there is a meeting that has ended by the time the meeting at `start_pointer` starts
            if startTime[start_pointer] >= endTime[end_pointer]:
                # Free up a room and increment the end_pointer.
                used_rooms -= 1
                end_pointer += 1

            # If a room got free, then this used_rooms += 1 wouldn't have any effect. used_rooms would
            # remain the same in that case. If no room was free, then this would increase used_rooms
            used_rooms += 1    
            start_pointer += 1   

        return used_rooms

#### [2402. Meeting Rooms III](https://leetcode.com/problems/meeting-rooms-iii)

In [34]:
from heapq import heappush, heappop, heapify
from typing import List

class Solution:
    def mostBooked(self, n: int, meetings: List[List[int]]) -> int:
        unusedRoom = [i for i in range(n)] #minheap for unusedroom bc we retrieve smallest roomNum
        usedRoom = [] #min heap for usedRoom currently in use, (end_time, room_index) bc retrieve smallest endtime when unusedroom == 0
        meeting_count = [0] * n
        
        # Process meetings in order of their start times
        for start, end in sorted(meetings): #M⋅logM for soting, M⋅logN for using n-sized heap looping M
            
            # Free up all rooms that (meetings that have ended by "start")
            while usedRoom and usedRoom[0][0] <= start:
                _, room = heappop(usedRoom)  
                heappush(unusedRoom, room)  
            
            # Assign a room to the current meeting
            if unusedRoom:
                # If there are available rooms, take the one with the smallest index
                room = heappop(unusedRoom)
                heappush(usedRoom, [end, room])  # Schedule the meeting in this room
            else:
                # get minEndtime from usedRooms for min delay.
                endTime, room = heappop(usedRoom)
                heappush(usedRoom, [end + (endTime - start), room])  
            
            # Increment the meeting count for the assigned room
            meeting_count[room] += 1
        
        # Return the index of the smallest room with the maximum number of meetings held
        return meeting_count.index(max(meeting_count))
#time/: O(M⋅logM+M⋅logN), space O(N+sort)

#### [1229. Meeting Scheduler](https://leetcode.com/problems/meeting-scheduler/)

In [ ]:
# using sorting
def minAvailableDuration(slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
    slots1.sort()
    slots2.sort()
    m, n = len(slots1), len(slots2)
    i = j = 0
    while i < m and j < n:
        intersectStart = max(slots1[i][0], slots2[j][0])
        intersectEnd = min(slots1[i][1], slots2[j][1])
        if intersectEnd - intersectStart >= duration:
            return [intersectStart, intersectStart + duration]
        if slots1[i][1] < slots2[j][1]:
            i += 1
        else:
            j += 1
    return []

In [ ]:
'''
Initialize a heap timeslots and push time slots that last longer than duration into it.
Iterate until there's only one time slot remaining in timeslots:
Pop the first time slot [start1, end1] from timeslots.
Retrieve the next time slot [start2, end2], which is the current top element in timeslots.
If we find end1 >= start2 + duration, because start1 <= start2, the common slot is longer than duration and we can return it.
If we don't find the common slot that is longer than duration, return an empty array.
'''
class Solution:
    def minAvailableDuration(self, slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
        # build up a heap containing time slots last longer than duration
        timeslots = list(filter(lambda x: x[1] - x[0] >= duration, slots1 + slots2))
        heapq.heapify(timeslots)

        while len(timeslots) > 1:
            start1, end1 = heapq.heappop(timeslots)
            start2, end2 = timeslots[0]
            if end1 >= start2 + duration:
                return [start2, start2 + duration]
        return []
"""
Time : O((M+N)log(M+N)), when M is length of slots1 and N is length of slots2.
There are two parts to be analyzed: 1) building up the heap; 2) the iteration when we keep popping elements from the heap.
Space : O(M+N)"""

In [ ]:
#1999. Smallest Greater Multiple Made of Two Digits
from collections import deque
class Solution:
    def findInteger(self, k: int, digit1: int, digit2: int) -> int:
        q = deque()
        d1 = min(digit1,digit2)
        d2 = max(digit1,digit2)
        if d1!= 0:
            q.append(d1)
        if d2!= 0:
            q.append(d2)
        maxInt = 2**31-1
        while(q):
            nextInteger = q.popleft()
            if nextInteger > maxInt: return -1
            elif nextInteger > k and nextInteger % k == 0: return nextInteger

            q.append(nextInteger*10+d1)
            q.append(nextInteger*10+d2)
        return -1

        